## Load Data and Packages

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import pickle
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy.random import choice
import seaborn as sns

In [ ]:
## Load InMAP files -- see https://inmap.run/blog/2019/04/20/sr/ for details

from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from builtins import *

import warnings
warnings.filterwarnings("ignore",category = FutureWarning)
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

from io import BytesIO, TextIOWrapper
from zipfile import ZipFile
import urllib.request
import csv
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt


# Ensure compatibility between python 2 and python 3
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from builtins import *

import requests
import platform
import os
import stat
import tempfile
import json
import time
import subprocess
import geopandas as gpd
import shutil

def _download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = requests.get(url)
        # write to file
        file.write(response.content)

_inmap_exe = None
_tmpdir = tempfile.TemporaryDirectory()


if _inmap_exe == None:
    ost = platform.system()
    print("Downloading InMAP executable for %s"%ost, end='\r')
    if ost == "Windows":
        _inmap_exe = os.path.join(_tmpdir.name, "inmap_1.7.2.exe")
        _download("https://github.com/spatialmodel/inmap/releases/download/v1.7.2/inmap1.7.2windows-amd64.exe", _inmap_exe)
    elif ost == "Darwin":
        _inmap_exe = os.path.join(_tmpdir.name, "inmap_1.7.2")
        _download("https://github.com/spatialmodel/inmap/releases/download/v1.7.2/inmap1.7.2darwin-amd64", _inmap_exe)
    elif ost == "Linux":
        _inmap_exe = os.path.join(_tmpdir.name, "inmap_1.7.2")
        _download("https://github.com/spatialmodel/inmap/releases/download/v1.7.2/inmap1.7.2linux-amd64", _inmap_exe)
    else:
        raise(OSError("invalid operating system %s"%(ost)))
    os.chmod(_inmap_exe, stat.S_IXUSR|stat.S_IRUSR|stat.S_IWUSR)







def run_sr(emis, model, output_variables, emis_units="tons/year"):
    """
    Run the provided emissions through the specified SR matrix, calculating the
    specified output properties.

    Args:
        emis: The emissions to be calculated, Needs to be a geopandas dataframe.

        model: The SR matrix to use. Allowed values:
            isrm: The InMAP SR matrix
            apsca_q0: The APSCA SR matrix, annual average
            apsca_q1: The APSCA SR matrix, Jan-Mar season
            apsca_q2: The APSCA SR matrix, Apr-Jun season
            apsca_q3: The APSCA SR matrix, Jul-Sep season
            apsca_q4: The APSCA SR matrix, Oct-Dec season

        output_variables: Output variables to be calculated. See
            https://inmap.run/docs/results/ for more information.

        emis_units: The units that the emissions are in. Allowed values:
            'tons/year', 'kg/year', 'ug/s', and 'μg/s'.
    """


    global _tmpdir
    global _inmap_exe

    model_paths = {
        #"isrm": "/data/isrmv121/isrm_v1.2.1.ncf",
        "isrm": "/Users/norahennessy/Desktop/Stanford Research/IMSR/isrm_v1.2.1.ncf",
        "apsca_q0": "/data/apsca/apsca_sr_Q0_v1.2.1.ncf",
        "apsca_q1": "/data/apsca/apsca_sr_Q1_v1.2.1.ncf",
        "apsca_q2": "/data/apsca/apsca_sr_Q2_v1.2.1.ncf",
        "apsca_q3": "/data/apsca/apsca_sr_Q3_v1.2.1.ncf",
        "apsca_q4": "/data/apsca/apsca_sr_Q4_v1.2.1.ncf",
    }
    if model not in model_paths.keys():
        models = ', '.join("{!s}".format(k) for (k) in model_paths.keys())
        msg = 'model must be one of \{{!s}\}, but is `{!s}`'.format(models, model)
        raise ValueError(msg)
    model_path = model_paths[model]

    start = time.time()
    job_name = "run_aqm_%s"%start
    emis_file = os.path.join(_tmpdir.name, "%s.shp"%(job_name))
    emis.to_file(emis_file)
    out_file = os.path.join(_tmpdir.name, "%s_out.shp"%(job_name))
    
    subprocess.check_output([_inmap_exe, "srpredict",
            "--EmissionUnits=%s"%emis_units,
            "--EmissionsShapefiles=%s"%emis_file,
            "--OutputFile=%s"%out_file,
            "--OutputVariables=%s"%json.dumps(output_variables),
            "--SR.OutputFile=%s"%model_path])
    output = gpd.read_file(out_file)
    os.remove(out_file)
    return output


In [ ]:
def create_emissions_file(df):
    df = df.drop(columns = "geometry")
    df = df.rename(columns = {"centroid":"geometry"})
    df.loc[:,"height"] = 0.0
    df = df[["SOx","NOx","NH3","VOC","PM2_5","height","geometry"]]
    df = df.fillna(0)
    return df
    


def run_inmap(emissions_file):
    output_variables = {
        'TotalPM25':'PrimaryPM25 + pNH4 + pSO4 + pNO3 + SOA',
        'deathsK':'(exp(log(1.06)/10 * TotalPM25) - 1) * TotalPop * 1.06115917 * MortalityRate / 100000 * 1.036144578',
        'deathsL':'(exp(log(1.14)/10 * TotalPM25) - 1) * TotalPop * 1.06115917 * MortalityRate / 100000 * 1.036144578',
        'Population': 'TotalPop * 1.06115917',
        'Mortality': 'MortalityRate * 1.036144578',
        'deathsK_pc': 'deathsK/Population',
        'deathsL_pc': 'deathsL/Population'
    }
    
    resultsISRM = run_sr(emissions_file, model="isrm", emis_units="kg/year", output_variables=output_variables)
    return resultsISRM

## Preprocess Data

In [ ]:
county_emfac_vmt = pd.read_csv("County_number_vehicles/EMFAC_vmt_emisisons_2019_count.csv", skiprows = 8)

In [ ]:
county_emfac_vmt["veh_type"] = "none"
county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(["LHD1"]),"veh_type"]="LHD1"
county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(["LHD2"]),"veh_type"]="LHD2"
county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin([
       'T6 Instate Delivery Class 4', 'T6 Instate Delivery Class 5',
       'T6 Instate Delivery Class 6', 'T6 Instate Delivery Class 7',
       'T6 Instate Other Class 4', 'T6 Instate Other Class 5',
       'T6 Instate Other Class 6', 'T6 Instate Other Class 7',
       'T6 Instate Tractor Class 6', 'T6 Instate Tractor Class 7','T6 Public Class 4', 'T6 Public Class 5',
       'T6 Public Class 6', 'T6 Public Class 7', 'T6 Utility Class 5',
       'T6 Utility Class 6', 'T6 Utility Class 7', 'T6TS']),"veh_type"]="T6"

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['T6 CAIRP Class 4',
       'T6 CAIRP Class 5', 'T6 CAIRP Class 6', 'T6 CAIRP Class 7',
       'T6 OOS Class 4', 'T6 OOS Class 5', 'T6 OOS Class 6',
       'T6 OOS Class 7']),"veh_type"]="T6_OOS"

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['T7 CAIRP Class 8', 
                                                                'T7 NNOOS Class 8', 'T7 NOOS Class 8']),"veh_type"]="T7_OOS"

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['T7 Other Port Class 8', 'T7 POAK Class 8', 
                                                                'T7 POLA Class 8']),"veh_type"]="T7_Port"

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['T7 Public Class 8', 'T7 Single Concrete/Transit Mix Class 8',
       'T7 Single Dump Class 8', 'T7 Single Other Class 8',
       'T7 SWCV Class 8', 'T7 Tractor Class 8', 'T7 Utility Class 8',
       'T7IS']),"veh_type"]="T7"

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['MH']),"veh_type"]='MH'

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['Motor Coach']),"veh_type"]='MC'

county_emfac_vmt.loc[county_emfac_vmt["Vehicle Category"].isin(['All Other Buses','OBUS', 'SBUS','UBUS']),"veh_type"]='Buses'


In [ ]:
county_emfac_vmt = county_emfac_vmt[county_emfac_vmt.veh_type.isin(["LHD1","LHD2","MH","MC","Buses","T6","T6_OOS","T7","T7_OOS","T7_Port"])]

In [ ]:
county_vmt = county_emfac_vmt.groupby(["Region","veh_type"]).agg({"Population":"sum","Total VMT":"sum"}).reset_index()

In [ ]:
states = gpd.read_file("tl_2019_us_state/tl_2019_us_state.shp")
ct = gpd.read_file("tl_2019_us_county/tl_2019_us_county.shp")
ct = ct[ct.STATEFP=="06"]

In [ ]:
ct_vmt = ct.merge(county_vmt, left_on = "NAME", right_on = "Region", how = "right")

In [ ]:
total_vmt = county_emfac_vmt.groupby("veh_type").agg({"Population":"sum","Total VMT":"sum"}).reset_index()

In [ ]:
county_vmt["vmt_pct"] = 0
for v in total_vmt.veh_type.unique():
    for c in county_vmt.Region.unique():
        county_vmt.loc[(county_vmt.veh_type==v) & (county_vmt.Region==c),"vmt_pct"] = (
            county_vmt.loc[(county_vmt.veh_type==v) & (county_vmt.Region==c),"Total VMT"].sum()/
        total_vmt.loc[total_vmt.veh_type==v,"Total VMT"].sum()*100)

In [ ]:
county_pop = pd.read_csv("county_number_vehicles/FleetDB-Statewide-2019-All-GVWR-All-Agg-Agg-Agg-Agg-ByCounty.csv", skiprows = 13)
county_pop["veh_type"]= "none"
county_pop.loc[county_pop["Vehicle Category"].isin(["B","BT","BS"]),"veh_type"] = "Buses"
county_pop.loc[county_pop["Vehicle Category"]=="MC","veh_type"] = "MC"
county_pop.loc[county_pop["Vehicle Category"]=="MH","veh_type"] = "MH"
county_pop.loc[county_pop["Vehicle Category"]=="T6","veh_type"] = "T6"
county_pop.loc[county_pop["Vehicle Category"]=="T7","veh_type"] = "T7"
county_pop.loc[county_pop["Vehicle Category"]=="T4","veh_type"] = "LHD1"
county_pop.loc[county_pop["Vehicle Category"]=="T5","veh_type"] = "LHD2"
county_pop = county_pop.groupby(["veh_type","County"]).agg({"Vehicle Population":"sum"}).reset_index()

In [ ]:
tract_vmt = {} #percentage of county vehicle population in each census tract
for c in county_vmt.Region.unique():
    tracts = pd.read_csv(f"Census_tracts_2019_grouped/FleetDB-County-{c.upper()}-2019-All-GVWR-All-Agg-Agg-Agg-Agg-ByCensusBlockGroupCode.csv", skiprows = 13)
    tracts = tracts.groupby(["Vehicle Category","Census Block Group Code"]).agg({"Vehicle Population":"sum"}).reset_index()
    tracts["veh_type"] = "none"
    tracts.loc[tracts["Vehicle Category"].isin(["B","BT","BS"]),"veh_type"] = "Buses"
    tracts.loc[tracts["Vehicle Category"]=="MC","veh_type"] = "MC"
    tracts.loc[tracts["Vehicle Category"]=="MH","veh_type"] = "MH"
    tracts.loc[tracts["Vehicle Category"]=="T6","veh_type"] = "T6"
    tracts.loc[tracts["Vehicle Category"]=="T7","veh_type"] = "T7"
    tracts.loc[tracts["Vehicle Category"]=="T4","veh_type"] = "LHD1"
    tracts.loc[tracts["Vehicle Category"]=="T5","veh_type"] = "LHD2"
    tracts = tracts[tracts.veh_type!="none"]
    tracts["pop_pct"] = 0
    tracts.loc[tracts.veh_type=="Buses","pop_pct"] = (tracts.loc[tracts.veh_type=="Buses","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type=="Buses") & (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    tracts.loc[tracts.veh_type=="MC","pop_pct"] = (tracts.loc[tracts.veh_type=="MC","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type=="MC")& (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    tracts.loc[tracts.veh_type=="MH","pop_pct"] = (tracts.loc[tracts.veh_type=="MH","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type=="MH")& (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    tracts.loc[tracts.veh_type=="T6","pop_pct"] = (tracts.loc[tracts.veh_type=="T6","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type.isin(["T6","T6_OOS"]))& (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    tracts.loc[tracts.veh_type=="T7","pop_pct"] = (tracts.loc[tracts.veh_type=="T7","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type.isin(["T7","T7_OOS","T7_Port"]))& (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    tracts.loc[tracts.veh_type=="LHD1","pop_pct"] = (tracts.loc[tracts.veh_type=="LHD1","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type.isin(["LHD1"]))& (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    tracts.loc[tracts.veh_type=="LHD2","pop_pct"] = (tracts.loc[tracts.veh_type=="LHD2","Vehicle Population"]/
                                                      county_pop.loc[(county_pop.veh_type.isin(["LHD2"]))& (county_pop.County==c.upper()),"Vehicle Population"].sum()*100)
    
    tract_vmt[c] = tracts.groupby(["veh_type","Census Block Group Code"]).agg({"Vehicle Population":"sum","pop_pct":"sum"}).reset_index()

In [ ]:
n_tracts = set()
for k in tract_vmt.keys():
    n_tracts.update(tract_vmt[k]["Census Block Group Code"].unique())

In [ ]:
block_group_vmt_pcts = pd.DataFrame(columns = ["Census Block Group Code", "County","veh_type", "total_vmt_frac"])
for c in tract_vmt.keys():
    #Buses
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="Buses"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="Buses"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    #MH
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="MH"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="MH"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #MC
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="MC"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="MC"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #LHD1
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="LHD1"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="LHD1"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #LHD2
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="LHD2"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="LHD2"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #T6
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="T6"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="T6"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #T6 OOS
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="T6_OOS"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="T6"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    tract_vmt_pct["veh_type"] = "T6_OOS"
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #T7
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="T7"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="T7"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #T7 OOS
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="T7_OOS"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="T7"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    tract_vmt_pct["veh_type"] = "T7_OOS"
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])
    
    
    #T7 Port
    c_pct = county_vmt.loc[(county_vmt.Region==c) & (county_vmt.veh_type=="T7_Port"),"vmt_pct"].sum()/100
    tract_vmt_pct = tract_vmt[c].loc[tract_vmt[c]["veh_type"]=="T7"].copy()
    tract_vmt_pct["total_vmt_frac"] = tract_vmt_pct["pop_pct"]*c_pct/100
    tract_vmt_pct["County"] = c
    tract_vmt_pct["veh_type"] = "T7_Port"
    block_group_vmt_pcts = pd.concat([block_group_vmt_pcts, tract_vmt_pct[["Census Block Group Code","County","veh_type","total_vmt_frac"]]])


In [ ]:
block_group_vmt_pcts.to_csv("block_group_vmt_pct_2019.csv")

In [ ]:
emfac_data = pd.read_csv("County_number_vehicles/EMFAC_2019_my_emissions.csv", skiprows = 8)

In [ ]:
emfac_data["veh_type"] = "none"
emfac_data.loc[emfac_data["Vehicle Category"].isin(["LHD1"]),"veh_type"]="LHD1"
emfac_data.loc[emfac_data["Vehicle Category"].isin(["LHD2"]),"veh_type"]="LHD2"
emfac_data.loc[emfac_data["Vehicle Category"].isin([
       'T6 Instate Delivery Class 4', 'T6 Instate Delivery Class 5',
       'T6 Instate Delivery Class 6', 'T6 Instate Delivery Class 7',
       'T6 Instate Other Class 4', 'T6 Instate Other Class 5',
       'T6 Instate Other Class 6', 'T6 Instate Other Class 7',
       'T6 Instate Tractor Class 6', 'T6 Instate Tractor Class 7','T6 Public Class 4', 'T6 Public Class 5',
       'T6 Public Class 6', 'T6 Public Class 7', 'T6 Utility Class 5',
       'T6 Utility Class 6', 'T6 Utility Class 7', 'T6TS']),"veh_type"]="T6"

emfac_data.loc[emfac_data["Vehicle Category"].isin(['T6 CAIRP Class 4',
       'T6 CAIRP Class 5', 'T6 CAIRP Class 6', 'T6 CAIRP Class 7',
       'T6 OOS Class 4', 'T6 OOS Class 5', 'T6 OOS Class 6',
       'T6 OOS Class 7']),"veh_type"]="T6_OOS"

emfac_data.loc[emfac_data["Vehicle Category"].isin(['T7 CAIRP Class 8', 
                                                                'T7 NNOOS Class 8', 'T7 NOOS Class 8']),"veh_type"]="T7_OOS"

emfac_data.loc[emfac_data["Vehicle Category"].isin(['T7 Other Port Class 8', 'T7 POAK Class 8', 
                                                                'T7 POLA Class 8']),"veh_type"]="T7_Port"

emfac_data.loc[emfac_data["Vehicle Category"].isin(['T7 Public Class 8', 'T7 Single Concrete/Transit Mix Class 8',
       'T7 Single Dump Class 8', 'T7 Single Other Class 8',
       'T7 SWCV Class 8', 'T7 Tractor Class 8', 'T7 Utility Class 8',
       'T7IS']),"veh_type"]="T7"

emfac_data.loc[emfac_data["Vehicle Category"].isin(['MH']),"veh_type"]='MH'

emfac_data.loc[emfac_data["Vehicle Category"].isin(['Motor Coach']),"veh_type"]='MC'

emfac_data.loc[emfac_data["Vehicle Category"].isin(['All Other Buses','OBUS', 'SBUS','UBUS']),"veh_type"]='Buses'


In [ ]:
emfac_data_grouped = emfac_data.groupby(["veh_type","Model Year", "Fuel"]).agg({"Total VMT":"sum", 'Fuel Consumption':"sum", 'NOx_TOTEX':"sum",
                                                                               'PM2.5_TOTEX':"sum", 'ROG_TOTEX':"sum",'SOx_TOTEX':"sum", 
                                                                                'NH3_RUNEX':"sum" }).reset_index()

In [ ]:
emfac_data_grouped["NOx_g_gal"] = emfac_data_grouped["NOx_TOTEX"]*907185/emfac_data_grouped["Fuel Consumption"]/1000
emfac_data_grouped["PM2.5_g_gal"] = emfac_data_grouped["PM2.5_TOTEX"]*907185/emfac_data_grouped["Fuel Consumption"]/1000
emfac_data_grouped["ROG_g_gal"] = emfac_data_grouped["ROG_TOTEX"]*907185/emfac_data_grouped["Fuel Consumption"]/1000
emfac_data_grouped["SOx_g_gal"] = emfac_data_grouped["SOx_TOTEX"]*907185/emfac_data_grouped["Fuel Consumption"]/1000
emfac_data_grouped["NH3_g_gal"] = emfac_data_grouped["NH3_RUNEX"]*907185/emfac_data_grouped["Fuel Consumption"]/1000

In [ ]:
emfac_data_grouped =emfac_data_grouped[emfac_data_grouped.veh_type!="none"]

In [ ]:
emfac_data_grouped = emfac_data_grouped.drop(columns = ["NOx_TOTEX","PM2.5_TOTEX", "ROG_TOTEX", "NH3_RUNEX", "SOx_TOTEX"])

In [ ]:
for y in np.arange(2021, 2046):
    for v in emfac_data_grouped.veh_type.unique():
        for f in ["Diesel","Natural Gas","Gasoline"]:
            emfac_data_grouped = emfac_data_grouped.append(pd.DataFrame({"veh_type":v, "Model Year":y, 
                                                                         "Fuel":f, "Total VMT":0, "Fuel Consumption":0,
                                                                        "NOx_g_gal": emfac_data_grouped.loc[(emfac_data_grouped.veh_type==v) &
                                                                                                           (emfac_data_grouped["Model Year"]==2020)&
                                                                                                           (emfac_data_grouped["Fuel"]==f),"NOx_g_gal"],
                                                                        "PM2.5_g_gal": emfac_data_grouped.loc[(emfac_data_grouped.veh_type==v) &
                                                                                                           (emfac_data_grouped["Model Year"]==2020)&
                                                                                                           (emfac_data_grouped["Fuel"]==f),"PM2.5_g_gal"],
                                                                        "ROG_g_gal": emfac_data_grouped.loc[(emfac_data_grouped.veh_type==v) &
                                                                                                           (emfac_data_grouped["Model Year"]==2020)&
                                                                                                           (emfac_data_grouped["Fuel"]==f),"ROG_g_gal"],
                                                                        "SOx_g_gal": emfac_data_grouped.loc[(emfac_data_grouped.veh_type==v) &
                                                                                                           (emfac_data_grouped["Model Year"]==2020)&
                                                                                                           (emfac_data_grouped["Fuel"]==f),"SOx_g_gal"],
                                                                        "NH3_g_gal": emfac_data_grouped.loc[(emfac_data_grouped.veh_type==v) &
                                                                                                           (emfac_data_grouped["Model Year"]==2020)&
                                                                                                           (emfac_data_grouped["Fuel"]==f),"NH3_g_gal"]}))

In [ ]:
emfac_data_grouped.to_csv("emfac_efs_2019.csv")

## Read in preprocessed data and functions

In [ ]:
bg_vmt_pct = pd.read_csv("block_group_vmt_pct_2019.csv")
efs = pd.read_csv("emfac_efs_2019.csv")
bg = gpd.read_file("Census_tract_2019/tl_2019_06_bg/tl_2019_06_bg.shp")

In [ ]:
elec_co2_efs = pd.read_csv("electricity_co2_efs.csv")

In [ ]:
demographics = gpd.read_file("inmap_demographics.shp")
dem_colnames = ['ID', 'White_pct', 'Black_pct', 'Native_pct', 'Asian_pct',
       'Pac Islander_pct', 'Two or More_pct', 'Latino_pct',
       'White non-Latino_pct', 'Black non-Latino_pct', 'Asian non-Latino_pct',
       '<10,000', '10,000-14,999', '15,000 - 24,999', '25,000 - 34,999',
       '35,000 - 49,999', '50,000 - 74,999', '75,000 - 99,999',
       '100,000 - 149,999', '150,000 - 199,999', '>200,000',
       'Total Households', 'Median Income', 'index_right', 'REGION',
       'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME', 'LSAD',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON','geometry']
demographics.columns = dem_colnames
demographics["Region"] = ""
demographics.loc[demographics.DIVISION=='1',"Region"] = "New England"
demographics.loc[demographics.DIVISION=='2',"Region"] = "Middle Atlantic"
demographics.loc[demographics.DIVISION=='3',"Region"] = "East North Central"
demographics.loc[demographics.DIVISION=='4',"Region"] = "West North Central"
demographics.loc[demographics.DIVISION=='5',"Region"] = "South Atlantic"
demographics.loc[demographics.DIVISION=='6',"Region"] = "East South Central"
demographics.loc[demographics.DIVISION=='7',"Region"] = "West South Central"
demographics.loc[demographics.DIVISION=='8',"Region"] = "Mountain"
demographics.loc[demographics.DIVISION=='9',"Region"] = "Pacific"

demographics["Big Region"] = ""
demographics.loc[demographics.DIVISION=='1',"Big Region"] = "Northeast"
demographics.loc[demographics.DIVISION=='2',"Big Region"] = "Northeast"
demographics.loc[demographics.DIVISION=='3',"Big Region"] = "Midwest"
demographics.loc[demographics.DIVISION=='4',"Big Region"] = "Midwest"
demographics.loc[demographics.DIVISION=='5',"Big Region"] = "South"
demographics.loc[demographics.DIVISION=='6',"Big Region"] = "South"
demographics.loc[demographics.DIVISION=='7',"Big Region"] = "South"
demographics.loc[demographics.DIVISION=='8',"Big Region"] = "West"
demographics.loc[demographics.DIVISION=='9',"Big Region"] = "West"

In [ ]:
def attach_demographics(df, demographics):
    df = df.to_crs(4326)
    result = df.merge(demographics, how = "inner", left_index = True, right_index = True)
    for c in ['White_pct', 'Black_pct', 'Native_pct', 'Asian_pct',
       'Pac Islander_pct', 'Two or More_pct', 'Latino_pct',
       'White non-Latino_pct', 'Black non-Latino_pct', 'Asian non-Latino_pct',
       '<10,000', '10,000-14,999', '15,000 - 24,999', '25,000 - 34,999',
       '35,000 - 49,999', '50,000 - 74,999', '75,000 - 99,999',
       '100,000 - 149,999', '150,000 - 199,999', '>200,000']:
        result[f"{c}_deathsK"] = result["deathsK"] * result[c]/100
        result[f"{c}_pop"] = result[c]/100*result["Population"]
        result[f"{c}_deathsK_pc"] = result[f"{c}_deathsK"]/result[f"{c}_pop"]*100000
    result = result.set_geometry("geometry_x", drop = True)
    result = result.drop(columns = ["geometry_y"])
    
    return result

In [ ]:
def get_cum_co2(model):
    years = np.arange(model.year.min(), model.year.max()+1)
    cum_co2 = pd.DataFrame(columns = ["cum_co2"], index = years)
    for y in years:
        cum_co2.loc[y,"cum_co2"] = model.loc[model.year <= y,"co2"].sum()
        
    return cum_co2

In [ ]:
def add_used_vehicle_value(model):
    used_prices = pd.read_excel("Used HDV Prices.xlsx")
    model["used_value"] = 0
    for v in model.veh_type.unique():
        for a in model.loc[model.veh_type==v,"age"].unique():
            model.loc[(model.veh_type==v) & (model.age == a), "used_value"] = (used_prices.loc[used_prices.Age==a,v].item()*
                                                                       model.loc[(model.veh_type==v) & (model.age == a), "early_ret"])
    return model

In [ ]:
def add_new_veh_value(model):
    new_price = pd.read_csv("hdv_bev_prices.csv")
    model["new_value"] = 0
    for v in model.veh_type.unique():
        for a in model.loc[model.veh_type==v,"age"].unique():
            model.loc[(model.veh_type==v) & (model.age == a), "new_value"] = (new_price.loc[new_price.veh_type==v,"price"].item()*
                                                                       model.loc[(model.veh_type==v), "early_ret"])

In [ ]:
def get_criteria_emissions(model):
    model["NOX"] = model["fuel_consumption"]*model["NOx_g_gal"]
    model["PM25"] = model["fuel_consumption"]*model["PM2.5_g_gal"]
    model["VOC"] = model["fuel_consumption"]*model["ROG_g_gal"]
    model["SO2"] = model["fuel_consumption"]*model["SOx_g_gal"]
    model["NH3"] = model["fuel_consumption"]*model["NH3_g_gal"]
    return model

## Read in Stock and Flow Model Results

In [ ]:
veh_types = ["T6", "T6_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T7","T7_OOS"]
BAU = {}
for v in veh_types:
    BAU[v] = pd.read_csv(f"/Users/norahennessy/Documents/GitHub/Stock-and-Flow/Paper_results_new/model_{v}_BAU.csv")

BAU_all = BAU["T6"]
for v in ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port"]:
    BAU_all  = BAU_all.append(BAU[v])

In [ ]:
for y in elec_co2_efs.year.unique():
    BAU_all.loc[(BAU_all.fuel_type=="Electricity") & (BAU_all.year == y),"co2"] = (
        BAU_all.loc[(BAU_all.fuel_type=="Electricity") & (BAU_all.year == y),"fuel_consumption"]*
    elec_co2_efs.loc[elec_co2_efs.year==y,"CO2_g_kwh"].sum()*1e-6)

In [ ]:
BAU_all = BAU_all.merge(efs, how = "left", left_on = ["model_year","fuel_type", "veh_type"], right_on = ["Model Year", "Fuel","veh_type"])
BAU_all = get_criteria_emissions(BAU_all)

In [ ]:
#ZEV Sales (no retirements)
zev_years = [2025, 2030, 2035, 2040]
veh_types = ["T6","T6_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T7","T7_OOS"]
frac_elec = 1
input_file = "Inputs_mixed_scenario_no_early_ret_EMFAC_sales<1_2020_surv_updatedfe_vint_vmt_actual_surv.xlsx"

results = {}

for v in veh_types:
    results[v] = {}
    for z in zev_years:
        #results[v][z] = pd.read_csv(f"/Users/norahennessy/Documents/GitHub/Stock-and-Flow/EMFAC_sales_only_scenarios/model_{v}_{z}_1_sales<1_2020_surv_updatedfe_vint_vmt_actual_surv.csv")
        results[v][z] = pd.read_csv(f"/Users/norahennessy/Documents/GitHub/Stock-and-Flow/Paper_results_new/model_{v}_{z}.csv")

        add_used_vehicle_value(results[v][z])
        add_new_veh_value(results[v][z])


In [ ]:
results_all = {}
for z in zev_years:
    results_all[z] = results["T6"][z]
    for v in ["T7", "T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port"]:
        results_all[z] = results_all[z].append(results[v][z])
        for y in elec_co2_efs.year.unique():
            results_all[z].loc[(results_all[z].fuel_type=="Electricity") & (results_all[z].year == y),"co2"] = (
            results_all[z].loc[(results_all[z].fuel_type=="Electricity") & (results_all[z].year == y),"fuel_consumption"]*
            elec_co2_efs.loc[elec_co2_efs.year==y,"CO2_g_kwh"].sum()*1e-6) #tonnes
    results_all[z] = results_all[z].merge(efs, how = "left", left_on = ["model_year","fuel_type", "veh_type"], right_on = ["Model Year", "Fuel","veh_type"])
    results_all[z] = get_criteria_emissions(results_all[z])

In [ ]:
#ZEV Retirements
zev_years = [2035,2040]
veh_types = ["T6", "T6_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port","T7","T7_OOS"]
ret_ages = [5,10]
ret_years = [2025,2030,2035,2040]
frac_elec = 1
input_file = "Inputs_mixed_scenario_no_early_ret_EMFAC_sales<1_2020_surv_updatedfe_vint_vmt_actual_surv.xlsx"

results_ret = {}

for v in veh_types:
    results_ret[v] = {}
    for z in zev_years:
        results_ret[v][z] = {}
        for ra in ret_ages:
            results_ret[v][z][ra] = {}
            for ry in ret_years:
                #results_ret[v][z][ra][ry] = pd.read_csv(f"/Users/norahennessy/Documents/GitHub/Stock-and-Flow/EMFAC_retirement_and_sales_scenarios/model_{v}_{z}_zev_sales_overnight_{ra}_{ry}_1_sales<1_2020_surv_updatedfe_vint_vmt_actual_surv.csv")
                results_ret[v][z][ra][ry] = pd.read_csv(f"/Users/norahennessy/Documents/GitHub/Stock-and-Flow/Paper_results_new/model_{v}_{z}_{ra}_{ry}.csv")
                
                #Add Cumulative CO2 and cumulative veh_years_removed
                #get_veh_years_remaining(results_ret[v][z][ra][ry],input_file)
                #get_cum_veh_years_removed(results_ret[v][z][ra][ry])
                add_used_vehicle_value(results_ret[v][z][ra][ry])
                add_new_veh_value(results_ret[v][z][ra][ry])
                results_ret[v][z][ra][ry] = results_ret[v][z][ra][ry].merge(efs, how = "left", left_on = ["model_year","fuel_type", "veh_type"], right_on = ["Model Year", "Fuel","veh_type"])
                results_ret[v][z][ra][ry] = get_criteria_emissions(results_ret[v][z][ra][ry])
                # for v in ["T7_Port","T7","T7_OOS"]: 
#     results_ret[v] = {}
#     for z in zev_years:
#         results_ret[v][z] = {}
#         for ra in ret_ages:
#             results_ret[v][z][ra] = {}
#             for ry in ret_years:
#                 results_ret[v][z][ra][ry] = pd.read_csv(f"/Users/norahennessy/Documents/GitHub/Stock-and-Flow/EMFAC_retirement_and_sales_scenarios/model_{v}_{z}_zev_sales_overnight_{ra}_{ry}_1_sales<1_2020_surv_updatedfe_vint_vmt_actual_surv_t7port.csv")
#                 add_used_vehicle_value(results_ret[v][z][ra][ry])

In [ ]:
results_all_rets = {}
for z in zev_years:
    for ra in ret_ages:
        for ry in ret_years:
            results_all_rets[f"{z}-{ra}-{ry}"] = results_ret["T6"][z][ra][ry]
            for v in ["T7","T6_OOS","T7_OOS", "Buses","MH","MC","LHD1","LHD2","T7_Port"]:
                results_all_rets[f"{z}-{ra}-{ry}"]  = results_all_rets[f"{z}-{ra}-{ry}"] .append(results_ret[v][z][ra][ry])
                
                for y in elec_co2_efs.year.unique():
                    results_all_rets[f"{z}-{ra}-{ry}"] .loc[(results_all_rets[f"{z}-{ra}-{ry}"] .fuel_type=="Electricity") & (results_all_rets[f"{z}-{ra}-{ry}"] .year == y),"co2"] = (
                    results_all_rets[f"{z}-{ra}-{ry}"] .loc[(results_all_rets[f"{z}-{ra}-{ry}"] .fuel_type=="Electricity") & (results_all_rets[f"{z}-{ra}-{ry}"] .year == y),"fuel_consumption"]*
                    elec_co2_efs.loc[elec_co2_efs.year==y,"CO2_g_kwh"].sum()*1e-6)
                    

## Air Quality Analysis

In [ ]:
## Assign emissions to block groups:
base_year = 2019
end_year = 2045
emissions = {}
bg_emissions = {}
for v in veh_types:
    bg_emissions[v] = {}
    annual_emissions= pd.DataFrame(index = np.arange(base_year, end_year+1), columns = ["SOx","NOx","NH3","VOC","PM2_5"])
    for y in np.arange(base_year, end_year+1):        
        annual_emissions.loc[y,"NOx"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.veh_type==v),"NOX"].sum()/1000
        annual_emissions.loc[y,"VOC"] = BAU_all.loc[(BAU_all.year==y)& (BAU_all.veh_type==v),"VOC"].sum()/1000
        annual_emissions.loc[y,"PM2_5"] = BAU_all.loc[(BAU_all.year==y)& (BAU_all.veh_type==v),"PM25"].sum()/1000
        annual_emissions.loc[y,"SOx"] = BAU_all.loc[(BAU_all.year==y)& (BAU_all.veh_type==v),"SO2"].sum()/1000
        annual_emissions.loc[y,"NH3"] = BAU_all.loc[(BAU_all.year==y)& (BAU_all.veh_type==v),"NH3"].sum()/1000
    emissions[v] = annual_emissions
    
    for y in np.arange(base_year, end_year+1):
        bg_emissions[v][y] = bg_vmt_pct[bg_vmt_pct.veh_type==v].copy()
        bg_emissions[v][y]["NOx"] = annual_emissions.loc[y,"NOx"]*bg_emissions[v][y]["total_vmt_frac"]
        bg_emissions[v][y]["VOC"] = annual_emissions.loc[y,"VOC"]*bg_emissions[v][y]["total_vmt_frac"]
        bg_emissions[v][y]["PM2_5"] = annual_emissions.loc[y,"PM2_5"]*bg_emissions[v][y]["total_vmt_frac"]
        bg_emissions[v][y]["SOx"] = annual_emissions.loc[y,"SOx"]*bg_emissions[v][y]["total_vmt_frac"]
        bg_emissions[v][y]["NH3"] = annual_emissions.loc[y,"NH3"]*bg_emissions[v][y]["total_vmt_frac"]
 

In [ ]:
def assign_bg_emissions(model):
    base_year = 2019
    end_year = 2045
    emissions = {}
    bg_emissions = {}
    for v in veh_types:
        bg_emissions[v] = {}
        annual_emissions= pd.DataFrame(index = np.arange(base_year, end_year+1), columns = ["SOx","NOx","NH3","VOC","PM2_5","elec_consumption"])
        for y in np.arange(base_year, end_year+1):        
            annual_emissions.loc[y,"NOx"] = model.loc[(model.year==y) & (model.veh_type==v),"NOX"].sum()/1000
            annual_emissions.loc[y,"VOC"] = model.loc[(model.year==y)& (model.veh_type==v),"VOC"].sum()/1000
            annual_emissions.loc[y,"PM2_5"] = model.loc[(model.year==y)& (model.veh_type==v),"PM25"].sum()/1000
            annual_emissions.loc[y,"SOx"] = model.loc[(model.year==y)& (model.veh_type==v),"SO2"].sum()/1000
            annual_emissions.loc[y,"NH3"] = model.loc[(model.year==y)& (model.veh_type==v),"NH3"].sum()/1000
            annual_emissions.loc[y,"elec_consumption"] = model.loc[(model.year==y)& (model.veh_type==v) & (model.fuel_type=="Electricity"),"fuel_consumption"].sum()
        emissions[v] = annual_emissions

        for y in np.arange(base_year, end_year+1):
            bg_emissions[v][y] = bg_vmt_pct[bg_vmt_pct.veh_type==v].copy()
            bg_emissions[v][y]["NOx"] = annual_emissions.loc[y,"NOx"]*bg_emissions[v][y]["total_vmt_frac"]
            bg_emissions[v][y]["VOC"] = annual_emissions.loc[y,"VOC"]*bg_emissions[v][y]["total_vmt_frac"]
            bg_emissions[v][y]["PM2_5"] = annual_emissions.loc[y,"PM2_5"]*bg_emissions[v][y]["total_vmt_frac"]
            bg_emissions[v][y]["SOx"] = annual_emissions.loc[y,"SOx"]*bg_emissions[v][y]["total_vmt_frac"]
            bg_emissions[v][y]["NH3"] = annual_emissions.loc[y,"NH3"]*bg_emissions[v][y]["total_vmt_frac"]
            bg_emissions[v][y]["elec_consumption"] = annual_emissions.loc[y,"elec_consumption"]*bg_emissions[v][y]["total_vmt_frac"] #kwh
            
    return bg_emissions



In [ ]:
bg_emis = {}
bg_emis["BAU"] = assign_bg_emissions(BAU_all)
bg_emis["ZEV_mandate"] = {}
for z in [2025, 2030, 2035, 2040]:
    bg_emis["ZEV_mandate"][z] = assign_bg_emissions(results_all[z])
bg_emis["retirements"] = {}
for z in zev_years:
    for ra in ret_ages:
        for ry in ret_years:
            bg_emis["retirements"][f"{z}-{ra}-{ry}"] = assign_bg_emissions(results_all_rets[f"{z}-{ra}-{ry}"])


In [ ]:
for z in [2025, 2030]:
    bg_emis["ZEV_mandate"][z] = assign_bg_emissions(results_all[z])

In [ ]:
pm25_fossil = {}
for v in veh_types:
    print(v)
    pm25_fossil[v] = run_emissions(bg_emis["BAU"],v,2019)

In [ ]:
with open("pm25_twh_consumption_factors",'rb') as f:
    BA_pm25_twh = pickle.load(f)

In [ ]:
BAs = gpd.read_file("Control_Areas.shp")
BA_codes = pd.read_csv("ba_names.csv")
BA_codes["NAME"]=BA_codes["BANAME"].str.upper()
BAs=BAs.merge(BA_codes,how="left",left_on="NAME",right_on = "NAME")

In [ ]:
def get_ba_electricity(bg_emis):
    bg_emis_geo = bg.merge(bg_emis, how = "inner", left_on = "GEOID", right_on = "Census Block Group Code")
    bg_emis_geo["centroid"] = bg_emis_geo["geometry"].to_crs('+proj=cea').centroid.to_crs(4326)
    bg_emis_geo = bg_emis_geo.sjoin(BAs.to_crs(bg_emis_geo.crs), predicate = "within")

    count_dups = bg_emis_geo.groupby("GEOID").count()["STATEFP"]
    for index, row in bg_emis_geo.iterrows():
        lookup = row["GEOID"]
        n = count_dups[lookup]
        bg_emis_geo.loc[bg_emis_geo.GEOID == lookup,"total_vmt_frac"] = bg_emis_geo.loc[bg_emis_geo.GEOID == lookup,"total_vmt_frac"]/n
    ba_electricity = bg_emis_geo.groupby("BACODE").agg({"total_vmt_frac":"sum"})
        #Reassign consumpgion in GRIS to PNM (closest BA)
    if "GRIS" in ba_electricity.index:
        ba_electricity.loc["PNM","total_vmt_frac"] += ba_electricity.loc["GRIS","total_vmt_frac"]
        ba_electricity = ba_electricity.drop("GRIS")
        
    return ba_electricity


In [ ]:
def get_elec_pm25(ba_electricity,pm25_twh):
    BA_deathsK = {}
    for ba in ba_electricity.index:
        #print(ba)
        BA_deathsK[ba] = pm25_twh[ba]
        #print(BA_deathsK[ba])
        BA_deathsK[ba]["TotalPM25"] = BA_deathsK[ba]["pm25_twh"]*ba_electricity.loc[ba,"total_vmt_frac"]
    

    
    BAs = list(BA_deathsK.keys())
    #print("BAs[0]:",BAs[0])
    total_BA_deathsK = BA_deathsK[BAs[0]].copy()
    total_BA_deathsK["TotalPM25"] = 0
    for ba in BAs[0:]:
        #print(ba)
        total_BA_deathsK["TotalPM25"]+=BA_deathsK[ba]["TotalPM25"]
    total_BA_deathsK["TotalDeathsK"] = ((np.exp(np.log(1.06)/10 * 
                                                total_BA_deathsK["TotalPM25"]) - 1) * 
                                        total_BA_deathsK["Population"] * 
                                        total_BA_deathsK["Mortality"] / 100000 )
    total_BA_deathsK["deathsK"] = total_BA_deathsK["TotalDeathsK"]
    total_BA_deathsK["deathsK_pc"] = total_BA_deathsK["deathsK"]/total_BA_deathsK["Population"]
    return total_BA_deathsK

In [ ]:
pm25_elec = {}
for v in veh_types:
    ba_elec = get_ba_electricity(bg_emis["BAU"][v][2019])
    results_elec = get_elec_pm25(ba_elec, BA_pm25_twh)
    pm25_elec[v] = results_elec

In [ ]:
races = ['White', 'Black', 'Native', 'Asian',
       'Pac Islander', 'Two or More', 'Latino',
       'White non-Latino', 'Black non-Latino', 'Asian non-Latino']
incomes = ['<10,000', '10,000-14,999', '15,000 - 24,999', '25,000 - 34,999',
       '35,000 - 49,999', '50,000 - 74,999', '75,000 - 99,999',
       '100,000 - 149,999', '150,000 - 199,999', '>200,000']
results = {}
results["BAU"] = {}
for v in veh_types:
    print(v)
    results["BAU"][v] = {}
    results["BAU"][v]["Fossil"] = pd.DataFrame(index = np.arange(2019, 2046), columns = 
                                               ["deathsK_CA", "deathsK_OCA", "deathsK_pcCA"])
    tot_emissions = pd.DataFrame(index = np.arange(2019, 2046), columns = ["NOx"])
    for y in np.arange(2019,2046):
        tot_emissions.loc[y,"NOx"] = bg_emis["BAU"][v][y]["NOx"].sum()
    tot_emissions["pct"] = tot_emissions["NOx"]/tot_emissions.loc[2019,"NOx"]
    # print(tot_emissions)
    for y in np.arange(2019, 2046):   
        res = pm25_fossil[v].copy()
        res["deathsK"] = ((np.exp(np.log(1.06)/10 * res["TotalPM25"]*tot_emissions.loc[y,"pct"]) - 1) * 
    res["Population"]  * res["Mortality"] / 100000)
        res = attach_demographics(res, demographics)
        # print(res.head())
        results["BAU"][v]["Fossil"].loc[y, "deathsK_CA"] = res.loc[res.STATEFP=="06","deathsK"].sum()
        results["BAU"][v]["Fossil"].loc[y,"deathsK_OCA"] = res.loc[res.STATEFP!="06","deathsK"].sum()
        results["BAU"][v]["Fossil"].loc[y, "deathsK_pcCA"] = (res.loc[res.STATEFP=="06","deathsK"].sum())/(res.loc[res.STATEFP=="06","Population"].sum())*100000
        for r in races:
            results["BAU"][v]["Fossil"].loc[y, f"{r}_deathsK_pc"] = ((
                res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()/
            (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()*100000)           
        for i in incomes:
                        results["BAU"][v]["Fossil"].loc[y, f"{i}_deathsK_pc"] = ((
                res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",i]/100).sum()/
            (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",i]/100).sum()*100000) 

In [ ]:
results["ZEV_mandate"] = {}
for zy in [2025, 2030, 2035, 2040]:
    results["ZEV_mandate"][zy] = {}
    for v in veh_types:
        print(v)
        results["ZEV_mandate"][zy][v] = {}
        results["ZEV_mandate"][zy][v]["Fossil"] = pd.DataFrame(index = np.arange(2019, 2046), columns = 
                                                   ["deathsK_CA", "deathsK_OCA", "deathsK_pcCA"])
        tot_emissions = pd.DataFrame(index = np.arange(2019, 2046), columns = ["NOx"])
        for y in np.arange(2019,2046):
            tot_emissions.loc[y,"NOx"] = bg_emis["ZEV_mandate"][zy][v][y]["NOx"].sum()
        tot_emissions["pct"] = tot_emissions["NOx"]/tot_emissions.loc[2019,"NOx"]
        # print(tot_emissions)
        for y in np.arange(2019, 2046):   
            res = pm25_fossil[v].copy()
            res["deathsK"] = ((np.exp(np.log(1.06)/10 * res["TotalPM25"]*tot_emissions.loc[y,"pct"]) - 1) * 
        res["Population"]  * res["Mortality"] / 100000)
            res = attach_demographics(res, demographics)
            # print(res.head())
            results["ZEV_mandate"][zy][v]["Fossil"].loc[y, "deathsK_CA"] = res.loc[res.STATEFP=="06","deathsK"].sum()
            results["ZEV_mandate"][zy][v]["Fossil"].loc[y,"deathsK_OCA"] = res.loc[res.STATEFP!="06","deathsK"].sum()
            results["ZEV_mandate"][zy][v]["Fossil"].loc[y, "deathsK_pcCA"] = (res.loc[res.STATEFP=="06","deathsK"].sum())/(res.loc[res.STATEFP=="06","Population"].sum())*100000
            for r in races:
                results["ZEV_mandate"][zy][v]["Fossil"].loc[y, f"{r}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()*100000)           
            for i in incomes:
                            results["ZEV_mandate"][zy][v]["Fossil"].loc[y, f"{i}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",i]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",i]/100).sum()*100000) 

In [ ]:
results["retirements"] = {}
for zy in bg_emis["retirements"].keys():
    results["retirements"][zy] = {}
    for v in veh_types:
        print(v)
        results["retirements"][zy][v] = {}
        results["retirements"][zy][v]["Fossil"] = pd.DataFrame(index = np.arange(2019, 2046), columns = 
                                                   ["deathsK_CA", "deathsK_OCA", "deathsK_pcCA"])
        tot_emissions = pd.DataFrame(index = np.arange(2019, 2046), columns = ["NOx"])
        for y in np.arange(2019,2046):
            tot_emissions.loc[y,"NOx"] = bg_emis["retirements"][zy][v][y]["NOx"].sum()
        tot_emissions["pct"] = tot_emissions["NOx"]/tot_emissions.loc[2019,"NOx"]
        # print(tot_emissions)
        for y in np.arange(2019, 2046):   
            res = pm25_fossil[v].copy()
            res["deathsK"] = ((np.exp(np.log(1.06)/10 * res["TotalPM25"]*tot_emissions.loc[y,"pct"]) - 1) * 
        res["Population"]  * res["Mortality"] / 100000)
            res = attach_demographics(res, demographics)
            # print(res.head())
            results["retirements"][zy][v]["Fossil"].loc[y, "deathsK_CA"] = res.loc[res.STATEFP=="06","deathsK"].sum()
            results["retirements"][zy][v]["Fossil"].loc[y,"deathsK_OCA"] = res.loc[res.STATEFP!="06","deathsK"].sum()
            results["retirements"][zy][v]["Fossil"].loc[y, "deathsK_pcCA"] = (res.loc[res.STATEFP=="06","deathsK"].sum())/(res.loc[res.STATEFP=="06","Population"].sum())*100000
            for r in races:
                results["retirements"][zy][v]["Fossil"].loc[y, f"{r}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()*100000)           
            for i in incomes:
                            results["retirements"][zy][v]["Fossil"].loc[y, f"{i}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",i]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",i]/100).sum()*100000) 

In [ ]:
print(f"start_time = {time.time()}")
for v in veh_types:
    print(v)
    results["BAU"][v]["Elec"] = pd.DataFrame(index = np.arange(2019, 2046), columns = 
                                               ["deathsK_CA", "deathsK_OCA", "deathsK_pcCA"])
    
    elec_pct = pd.DataFrame(index = np.arange(2019, 2046), columns = ["pct"])
    elec_pct["pct"] = np.linspace(1,0,27)
    for y in np.arange(2019, 2046): 
        elec_cons = bg_emis["BAU"][v][y]["elec_consumption"].sum()/1e9
        res = pm25_elec[v].copy()
        # print(res["deathsK"].sum())
        res["deathsK"] = ((np.exp(np.log(1.06)/10 * res["TotalPM25"]*elec_pct.loc[y,"pct"]*elec_cons) - 1) * 
    res["Population"]  * res["Mortality"] / 100000)
        res = attach_demographics(res, demographics)
        # print(res["deathsK"].sum())
        results["BAU"][v]["Elec"].loc[y, "deathsK_CA"] = res.loc[res.STATEFP=="06","deathsK"].sum()
        results["BAU"][v]["Elec"].loc[y,"deathsK_OCA"] = res.loc[res.STATEFP!="06","deathsK"].sum()
        results["BAU"][v]["Elec"].loc[y, "deathsK_pcCA"] = (res.loc[res.STATEFP=="06","deathsK"].sum())/(res.loc[res.STATEFP=="06","Population"].sum())*100000
        for r in races:
            results["BAU"][v]["Elec"].loc[y, f"{r}_deathsK_pc"] = ((
                res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()/
            (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()*100000)           
        for i in incomes:
                        results["BAU"][v]["Elec"].loc[y, f"{i}_deathsK_pc"] = ((
                res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",i]/100).sum()/
            (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",i]/100).sum()*100000) 
print(f"end_time = {time.time()}")

In [ ]:
for zy in [2025, 2030, 2035, 2040]:
    for v in veh_types:
        print(v)
        results["ZEV_mandate"][zy][v]["Elec"] = pd.DataFrame(index = np.arange(2019, 2046), columns = 
                                                   ["deathsK_CA", "deathsK_OCA", "deathsK_pcCA"])
        elec_pct = pd.DataFrame(index = np.arange(2019, 2046), columns = ["pct"])
        elec_pct["pct"] = np.linspace(1,0,27)
        #print(elec_pct)
        for y in np.arange(2019, 2046): 
            elec_cons = bg_emis["ZEV_mandate"][zy][v][y]["elec_consumption"].sum()/1e9
            #print(elec_cons," twh, ",y)
            res = pm25_elec[v].copy()
        # print(res["deathsK"].sum())
            res["deathsK"] = ((np.exp(np.log(1.06)/10 * res["TotalPM25"]*elec_pct.loc[y,"pct"]*elec_cons) - 1) * 
    res["Population"]  * res["Mortality"] / 100000)
            #print(res["deathsK"].sum())
            res = attach_demographics(res, demographics)
            #print(res[res.STATEFP=="06"]["White_pct_deathsK"].sum())
        # print(res["deathsK"].sum())
            results["ZEV_mandate"][zy][v]["Elec"].loc[y, "deathsK_CA"] = res.loc[res.STATEFP=="06","deathsK"].sum()
            results["ZEV_mandate"][zy][v]["Elec"].loc[y,"deathsK_OCA"] = res.loc[res.STATEFP!="06","deathsK"].sum()
            results["ZEV_mandate"][zy][v]["Elec"].loc[y, "deathsK_pcCA"] = (res.loc[res.STATEFP=="06","deathsK"].sum())/(res.loc[res.STATEFP=="06","Population"].sum())*100000
            for r in races:
                results["ZEV_mandate"][zy][v]["Elec"].loc[y, f"{r}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()*100000)           
            for i in incomes:
                results["ZEV_mandate"][zy][v]["Elec"].loc[y, f"{i}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",i]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",i]/100).sum()*100000) 

In [ ]:
for zy in bg_emis["retirements"].keys():
    for v in veh_types:
        print(v)
        results["retirements"][zy][v]["Elec"] = pd.DataFrame(index = np.arange(2019, 2046), columns = 
                                                   ["deathsK_CA", "deathsK_OCA", "deathsK_pcCA"])
        elec_pct = pd.DataFrame(index = np.arange(2019, 2046), columns = ["pct"])
        elec_pct["pct"] = np.linspace(1,0,27)
        #print(elec_pct)
        for y in np.arange(2019, 2046): 
            elec_cons = bg_emis["retirements"][zy][v][y]["elec_consumption"].sum()/1e9
            #print(elec_cons," twh, ",y)
            res = pm25_elec[v].copy()
        # print(res["deathsK"].sum())
            res["deathsK"] = ((np.exp(np.log(1.06)/10 * res["TotalPM25"]*elec_pct.loc[y,"pct"]*elec_cons) - 1) * 
    res["Population"]  * res["Mortality"] / 100000)
            #print(res["deathsK"].sum())
            res = attach_demographics(res, demographics)
            #print(res[res.STATEFP=="06"]["White_pct_deathsK"].sum())
        # print(res["deathsK"].sum())
            results["retirements"][zy][v]["Elec"].loc[y, "deathsK_CA"] = res.loc[res.STATEFP=="06","deathsK"].sum()
            results["retirements"][zy][v]["Elec"].loc[y,"deathsK_OCA"] = res.loc[res.STATEFP!="06","deathsK"].sum()
            results["retirements"][zy][v]["Elec"].loc[y, "deathsK_pcCA"] = (res.loc[res.STATEFP=="06","deathsK"].sum())/(res.loc[res.STATEFP=="06","Population"].sum())*100000
            for r in races:
                results["retirements"][zy][v]["Elec"].loc[y, f"{r}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",f"{r}_pct"]/100).sum()*100000)           
            for i in incomes:
                results["retirements"][zy][v]["Elec"].loc[y, f"{i}_deathsK_pc"] = ((
                    res.loc[res.STATEFP=="06","deathsK"]*res.loc[res.STATEFP=="06",i]/100).sum()/
                (res.loc[res.STATEFP=="06","Population"]*res.loc[res.STATEFP=="06",i]/100).sum()*100000) 

# Plotting

In [ ]:
cum_emis = pd.DataFrame(index = ["BAU",2040,2035,2030,2025,
        "2040-10-2040","2040-5-2040",
        "2040-10-2035","2040-5-2035",
        "2040-10-2030","2040-5-2030",
        "2040-10-2025","2040-5-2025",
        "2035-10-2040","2035-5-2040",
        "2035-10-2035","2035-5-2035",
        "2035-10-2030","2035-5-2030",
        "2035-10-2025","2035-5-2025"], 
                       columns = ["cum_fossil_emis","cum_elec_emis","zev_year","ret_year","ret_age","color","outline_color","hatch"])

In [ ]:
cum_emis.loc["BAU","cum_fossil_emis"] = BAU_all.loc[BAU_all.fuel_type.isin(["Diesel","Nautral Gas","Gasoline"]),"co2"].sum()/1e6
cum_emis.loc["BAU","cum_elec_emis"] = BAU_all.loc[BAU_all.fuel_type.isin(["Electricity"]),"co2"].sum()/1e6
cum_emis.loc["BAU","zev_year"] = 100
cum_emis.loc["BAU","ret_year"] = 100
cum_emis.loc["BAU","ret_age"] = 100

for y in [2025, 2030, 2035, 2040]:
    cum_emis.loc[y,"cum_fossil_emis"] = results_all[y].loc[results_all[y]["fuel_type"].isin(["Diesel","Nautral Gas","Gasoline"]),"co2"].sum()/1e6
    cum_emis.loc[y,"cum_elec_emis"] = results_all[y].loc[results_all[y]["fuel_type"].isin(["Electricity"]),"co2"].sum()/1e6
    cum_emis.loc[y,"zev_year"] = y
    cum_emis.loc[y,"ret_year"] = 100
    cum_emis.loc[y,"ret_age"] = 100
for s in results_all_rets.keys():
    cum_emis.loc[s,"cum_fossil_emis"] = results_all_rets[s].loc[results_all_rets[s]["fuel_type"].isin(["Diesel","Nautral Gas","Gasoline"]),"co2"].sum()/1e6
    cum_emis.loc[s,"cum_elec_emis"] = results_all_rets[s].loc[results_all_rets[s]["fuel_type"].isin(["Electricity"]),"co2"].sum()/1e6
    cum_emis.loc[s,"zev_year"] = int(s.split("-")[0])
    cum_emis.loc[s,"ret_year"] = int(s.split("-")[2])
    cum_emis.loc[s,"ret_age"] = int(s.split("-")[1])

In [ ]:
cum_emis["ind_str"] = cum_emis.index
cum_emis["ind_str"] = cum_emis["ind_str"].astype("str")

In [ ]:
zy_cols = ["lightskyblue","paleturquoise","mediumturquoise","darkcyan"]#["gold","thistle","mediumturquoise","lightcoral"]
ry_cols = ["gold","darkorange","orangered","firebrick"]#["midnightblue","orange","brown","pink"]
ra_hatch = ["//","o"]

cum_emis.loc["BAU","color"] = "gray"
cum_emis.loc["BAU","hatch"] = ""
cum_emis.loc["BAU","outline_color"] = "gray"
for i,zy in enumerate([2025, 2030,2035,2040]):
    cum_emis.loc[cum_emis.zev_year==zy,"color"] = zy_cols[i]
    cum_emis.loc[cum_emis.zev_year==zy,"hatch"] = ""
    cum_emis.loc[cum_emis.zev_year==zy,"outline_color"] = zy_cols[i] 
for i,ry in enumerate([2025,2030,2035,2040]):
    print(i,ry)
    cum_emis.loc[cum_emis.ret_year==ry,"outline_color"] = ry_cols[i]
for i, ra in enumerate([5,10]):
    cum_emis.loc[cum_emis.ret_age==ra,"hatch"] = ra_hatch[i]


In [ ]:
df_mapping = pd.DataFrame({
    'scenario': ['BAU', 2040,2035,2030,2025,],
})

In [ ]:
cum_emis["tot_emis"] = cum_emis["cum_fossil_emis"]+cum_emis["cum_elec_emis"]

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
f, ax = plt.subplots(1,1, figsize = (20,8))


ax.bar(x= cum_emis.ind_str, height = cum_emis.cum_fossil_emis,color = cum_emis.color, 
       hatch = cum_emis.hatch, edgecolor = cum_emis.outline_color, linewidth = 3)
ax.bar(x= cum_emis.ind_str, height = cum_emis.cum_elec_emis,color = cum_emis.color, 
       hatch = cum_emis.hatch, edgecolor = cum_emis.outline_color, linewidth = 3, alpha = 0.5, bottom = cum_emis.cum_fossil_emis)
ax.vlines(0.5,0,1400, linestyles = "dashed", color = "black")
ax.vlines(4.5,0,1400, linestyles = "dashed", color = "black")
ax.set_ylim([0,1400])
plt.xticks(rotation = 45)
ax.text(-1,1300,"BAU", fontsize = 14)
ax.text(0.7,1300,"ZEV Mandate Scenarios", fontsize = 14)
ax.text(10,1300,"Accelerated Retirement Scenarios", fontsize = 14)
ax.set_ylabel("Cumulative CO2 Emissions \n(Million tonnes)", fontsize = 16)
ax.text(7,800,"ZEV Mandate in 2040", fontsize = 14)
ax.text(16,800,"ZEV Mandate in 2035", fontsize = 14)


handles = [
    mpatches.Patch(facecolor=zy_cols[0], hatch='', edgecolor=zy_cols[0], label = "2025"),
    mpatches.Patch(facecolor=zy_cols[1], hatch='', edgecolor=zy_cols[1], label = "2030"),
    mpatches.Patch(facecolor=zy_cols[2], hatch='', edgecolor=zy_cols[2], label = "2035"),
    mpatches.Patch(facecolor=zy_cols[3], hatch='', edgecolor=zy_cols[3], label = "2040")
]

handles2 = [
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[0], label = "2025"),
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[1], label = "2030"),
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[2], label = "2035"),
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[3], label = "2040")
]

handles3 = [
    mpatches.Patch(facecolor="white", hatch=ra_hatch[0], edgecolor="black", label = "5 years"),
    mpatches.Patch(facecolor="white", hatch=ra_hatch[1], edgecolor = "black", label = "10 years")
]

handles4 = [
    mpatches.Patch(facecolor="gray", hatch='', edgecolor="gray", label = "ICE Tailpipe Emissions"),
    mpatches.Patch(facecolor="gray", hatch='', edgecolor="gray", alpha = 0.5, label = "Electricity Generation Emissions")
]


zy_cols = ["lightskyblue","paleturquoise","mediumturquoise","darkcyan"]#["gold","thistle","mediumturquoise","lightcoral"]
ry_cols = ["gold","darkorange","orangered","firebrick"]#["midnightblue","orange","brown","pink"]
ra_hatch = ["//","o"]




# handles = [
#     mpatches.Patch(facecolor='red', hatch='/', edgecolor='black', label = "2025"),
#     mpatches.Patch(facecolor='green', hatch='\\', edgecolor='black'),
#     mpatches.Patch(facecolor='blue', hatch='|', edgecolor='black'),
#     mpatches.Patch(facecolor='yellow', hatch='-', edgecolor='black'),
#     plt.Line2D([], [], linestyle='dashed', color='black'),
#     plt.Line2D([], [], marker='', color='white', label='BAU'),
#     plt.Line2D([], [], marker='', color='white', label='ZEV Mandate Scenarios'),
#     plt.Line2D([], [], marker='', color='white', label='Accelerated Retirement Scenarios')
# ]

# Create the legend and add it to the plot
l1 = ax.legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 0.8), title = "ZEV Sales Mandate Year", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)
# l1.get_title().set_ha('left')
l2 = ax.legend(handles=handles2, loc='upper left', bbox_to_anchor=(1.02, 0.5), title = "Retirement Year", fancybox = False, fontsize = 14)
l2.get_title().set_fontsize(14)
# l2.get_title().set_ha('left')
l3 = ax.legend(handles=handles3, loc='upper left', bbox_to_anchor=(1.02, 0.2), title = "Retirement Age", fancybox = False, fontsize = 14)
l3.get_title().set_fontsize(14)
# l3.get_title().set_ha('left')
l4 = ax.legend(handles=handles4, loc='upper left', bbox_to_anchor=(1.02, 1), title = "Emissions Type", fancybox = False, fontsize = 14)
l4.get_title().set_fontsize(14)
# l4.get_title().set_ha('left')

ax.add_artist(l1)
ax.add_artist(l2)
ax.add_artist(l3)


for tick in ax.get_yticklabels():
    tick.set_fontsize(14)
for tick in ax.get_xticklabels():
    tick.set_fontsize(14)
    
ax.text(1, -325, "ZEV Mandate Year", fontsize = 14)
ax.text(8.5, -325,"ZEV mandate year - retirement age - retirement year", fontsize = 14)

plt.tight_layout()

# plt.gca().add_artist(l1)
# plt.gca().add_artist(l2)
#f.savefig("cumulative_emissions_barplot.png", bbox_inches = "tight")

In [ ]:
cum_deaths = pd.DataFrame(index = ["BAU",2040,2035,2030,2025,
        "2040-10-2040","2040-5-2040",
        "2040-10-2035","2040-5-2035",
        "2040-10-2030","2040-5-2030",
        "2040-10-2025","2040-5-2025",
        "2035-10-2040","2035-5-2040",
        "2035-10-2035","2035-5-2035",
        "2035-10-2030","2035-5-2030",
        "2035-10-2025","2035-5-2025"], 
                       columns = ["cum_deaths","zev_year","ret_year","ret_age","color","outline_color","hatch"])

In [ ]:
zev_mandate_emissions = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
for y in zev_mandate_emissions.index:
    zev_mandate_emissions.loc[y,"BAU"] = BAU_all.loc[BAU_all.year==y,"co2"].sum()/1e6
    for z in [2025,2030,2035,2040]:
        zev_mandate_emissions.loc[y,z] = results_all[z].loc[results_all[z].year==y,"co2"].sum()/1e6

In [ ]:
retirements_emissions = {}
for zy in [2035,2040]:
    retirements_emissions[zy] = {}
    for ra in [5,10]:
        retirements_emissions[zy][ra] = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
        for y in np.arange(2019, 2046):
            retirements_emissions[zy][ra].loc[y,"BAU"] = BAU_all.loc[BAU_all.year==y,"co2"].sum()/1e6
            for ry in [2025, 2030, 2035, 2040]:
                retirements_emissions[zy][ra].loc[y,ry] = results_all_rets[f"{zy}-{ra}-{ry}"].loc[results_all_rets[f"{zy}-{ra}-{ry}"]["year"]==y,"co2"].sum()/1e6
        

In [ ]:
zev_mandate_emissions_e = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
for y in zev_mandate_emissions_e.index:
    zev_mandate_emissions_e.loc[y,"BAU"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.fuel_type=="Electricity"),"co2"].sum()/1e6
    for z in [2025,2030,2035,2040]:
        zev_mandate_emissions_e.loc[y,z] = results_all[z].loc[(results_all[z].year==y) & (results_all[z].fuel_type=="Electricity"),"co2"].sum()/1e6

In [ ]:
retirements_emissions_e = {}
for zy in [2035,2040]:
    retirements_emissions_e[zy] = {}
    for ra in [5,10]:
        retirements_emissions_e[zy][ra] = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
        for y in np.arange(2019, 2046):
            retirements_emissions_e[zy][ra].loc[y,"BAU"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.fuel_type=="Electricity"),"co2"].sum()/1e6
            for ry in [2025, 2030, 2035, 2040]:
                retirements_emissions_e[zy][ra].loc[y,ry] = results_all_rets[f"{zy}-{ra}-{ry}"].loc[(results_all_rets[f"{zy}-{ra}-{ry}"]["year"]==y) &
                                                                                                    (results_all_rets[f"{zy}-{ra}-{ry}"]["fuel_type"]=="Electricity"),"co2"].sum()/1e6
   

In [ ]:
zev_mandate_emissions_t = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
for y in zev_mandate_emissions_t.index:
    zev_mandate_emissions_t.loc[y,"BAU"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.fuel_type.isin(["Diesel","Natural Gas", "Gasoline"])),"co2"].sum()/1e6
    for z in [2025,2030,2035,2040]:
        zev_mandate_emissions_t.loc[y,z] = results_all[z].loc[(results_all[z].year==y) & (results_all[z].fuel_type.isin(["Diesel","Natural Gas", "Gasoline"])),"co2"].sum()/1e6

In [ ]:
retirements_emissions_t = {}
for zy in [2035,2040]:
    retirements_emissions_t[zy] = {}
    for ra in [5,10]:
        retirements_emissions_t[zy][ra] = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
        for y in np.arange(2019, 2046):
            retirements_emissions_t[zy][ra].loc[y,"BAU"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.fuel_type.isin(["Diesel","Natural Gas", "Gasoline"])),"co2"].sum()/1e6
            for ry in [2025, 2030, 2035, 2040]:
                retirements_emissions_t[zy][ra].loc[y,ry] = results_all_rets[f"{zy}-{ra}-{ry}"].loc[(results_all_rets[f"{zy}-{ra}-{ry}"]["year"]==y) &
                                                                                                    (results_all_rets[f"{zy}-{ra}-{ry}"]["fuel_type"].isin(["Diesel","Natural Gas", "Gasoline"])),"co2"].sum()/1e6
  

In [ ]:
zy_cols = ["lightskyblue","paleturquoise","mediumturquoise","darkcyan"]#["gold","thistle","mediumturquoise","lightcoral"]
ry_cols = ["gold","darkorange","orangered","firebrick"]#["midnightblue","orange","brown","pink"]



In [ ]:
zev_mandate_stock = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
for y in zev_mandate_stock.index:
    zev_mandate_stock.loc[y,"BAU"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.fuel_type.isin(["Diesel","Natural Gas","Gasoline"])),"stock"].sum()/1e6
    for z in [2025,2030,2035,2040]:
        zev_mandate_stock.loc[y,z] = results_all[z].loc[(results_all[z].year==y) & (results_all[z].fuel_type.isin(["Diesel","Natural Gas","Gasoline"])),"stock"].sum()/1e6
        
retirements_stock = {}
for zy in [2035,2040]:
    retirements_stock[zy] = {}
    for ra in [5,10]:
        retirements_stock[zy][ra] = pd.DataFrame(index = np.arange(2019,2046), columns = ["BAU",2025,2030,2035,2040])
        for y in np.arange(2019, 2046):
            retirements_stock[zy][ra].loc[y,"BAU"] = BAU_all.loc[(BAU_all.year==y) & (BAU_all.fuel_type.isin(["Diesel","Natural Gas","Gasoline"])),"stock"].sum()/1e6
            for ry in [2025, 2030, 2035, 2040]:
                retirements_stock[zy][ra].loc[y,ry] = results_all_rets[f"{zy}-{ra}-{ry}"].loc[(results_all_rets[f"{zy}-{ra}-{ry}"]["year"]==y) & (results_all_rets[f"{zy}-{ra}-{ry}"]["fuel_type"].isin(["Diesel","Natural Gas","Gasoline"])),"stock"].sum()/1e6
  

In [ ]:
f, ax = plt.subplots(2,2, figsize = (15, 10))
ax = ax.flatten()

zev_mandate_stock.plot(ax = ax[0], color = ["black","lightskyblue","paleturquoise","mediumturquoise","darkcyan"], linewidth = 2)
retirements_stock[2035][5].plot(ax = ax[1], color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2)
retirements_stock[2035][10].plot(ax = ax[1], linestyle = "dashed", color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2)

ax[0].set_ylabel("Annual ICE Stock \n(number of vehicles)", fontsize = 14)
ax[0].set_title("A) ZEV Sales Mandate Scenarios\n ", fontsize = 16)
ax[1].set_title("B) Accelerated Retirement Scenarios\nZEV Mandate Year = 2035", fontsize = 16)
ax[0].grid()
ax[1].grid()


handles = [
    plt.Line2D([], [], linestyle='solid', color='black', label = "BAU", linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[0], label='2025', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[1], label='2030', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[2], label='2035', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[3], label='2040', linewidth = 2),    
]

handles2 = [
    plt.Line2D([], [], linestyle='solid', color='black', label = "BAU", linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[0], label='2025', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[1], label='2030', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[2], label='2035', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[3], label='2040', linewidth = 2),    
]

handles3 = [
    plt.Line2D([], [], marker='', color="gray", linestyle = "solid",label='5 years', linewidth = 2),
    plt.Line2D([], [], marker='', color=r"gray", linestyle = "dashed", label='10 years', linewidth = 2),
    
]


l0 = ax[0].legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), title = "ZEV Mandate Year", fancybox = False, fontsize = 14)
l0.get_title().set_fontsize(14)
l1 = ax[1].legend(handles=handles2, loc='upper left', bbox_to_anchor=(1.02, 1), title = "Retirement Year", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)
l2 = ax[1].legend(handles=handles3, loc='upper left', bbox_to_anchor=(1.02, 0.5), title = "Retirement Age", fancybox = False, fontsize = 14)
l2.get_title().set_fontsize(14)
ax[1].add_artist(l1)

for tick in ax[0].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[0].get_xticklabels():
    tick.set_fontsize(14)
for tick in ax[1].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[1].get_xticklabels():
    tick.set_fontsize(14)
ax[0].set_xlim([2019, 2045])
ax[1].set_xlim([2019, 2045])
ax[0].set_ylim([0,2.2])
ax[1].set_ylim([0,2.2])

zev_mandate_emissions.plot(ax = ax[2], color = ["black","lightskyblue","paleturquoise","mediumturquoise","darkcyan"], linewidth = 2, legend = False)
retirements_emissions[2035][5].plot(ax = ax[3], color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2, legend = False)
retirements_emissions[2035][10].plot(ax = ax[3], linestyle = "dashed", color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2, legend = False)

ax[2].set_ylabel("Annual CO2 Emissions \n(Million tonnes)", fontsize = 14)
ax[2].set_title("C) ZEV Sales Mandate Scenarios\n ", fontsize = 16)
ax[3].set_title("D) Accelerated Retirement Scenarios\nZEV Mandate Year = 2035", fontsize = 16)
ax[2].grid()
ax[3].grid()


handles = [
    plt.Line2D([], [], linestyle='solid', color='black', label = "BAU", linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[0], label='2025', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[1], label='2030', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[2], label='2035', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[3], label='2040', linewidth = 2),    
]

handles2 = [
    plt.Line2D([], [], linestyle='solid', color='black', label = "BAU", linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[0], label='2025', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[1], label='2030', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[2], label='2035', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[3], label='2040', linewidth = 2),    
]

handles3 = [
    plt.Line2D([], [], marker='', color="gray", linestyle = "solid",label='5 years', linewidth = 2),
    plt.Line2D([], [], marker='', color=r"gray", linestyle = "dashed", label='10 years', linewidth = 2),
    
]


# l0 = ax[2].legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), title = "ZEV Mandate Year", fancybox = False, fontsize = 14)
# l0.get_title().set_fontsize(14)
# l1 = ax[3].legend(handles=handles2, loc='upper left', bbox_to_anchor=(1.02, 1), title = "Retirement Year", fancybox = False, fontsize = 14)
# l1.get_title().set_fontsize(14)
# l2 = ax[2].legend(handles=handles3, loc='upper left', bbox_to_anchor=(1.02, 0.5), title = "Retirement Age", fancybox = False, fontsize = 14)
# l2.get_title().set_fontsize(14)
# ax[3].add_artist(l1)

for tick in ax[2].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[2].get_xticklabels():
    tick.set_fontsize(14)
for tick in ax[3].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[3].get_xticklabels():
    tick.set_fontsize(14)
ax[2].set_xlim([2019, 2045])
ax[3].set_xlim([2019, 2045])
ax[2].set_ylim([0,50])
ax[3].set_ylim([0,50])

plt.tight_layout()
#f.savefig("stock_and_emissions.png")

In [ ]:
f, ax = plt.subplots(2,2, figsize = (15, 10))
ax = ax.flatten()



zev_mandate_emissions_t.plot(ax = ax[0], color = ["black","lightskyblue","paleturquoise","mediumturquoise","darkcyan"], linewidth = 2, legend = False)
retirements_emissions_t[2035][5].plot(ax = ax[1], color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2, legend = False)
retirements_emissions_t[2035][10].plot(ax = ax[1], linestyle = "dashed", color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2, legend = False)

ax[0].set_ylabel("Annual ICE CO2 Emissions \n(Million tonnes)", fontsize = 14)
ax[0].set_title("A) ZEV Sales Mandate Scenarios\n ", fontsize = 16)
ax[1].set_title("B) Accelerated Retirement Scenarios\nZEV Mandate Year = 2035", fontsize = 16)
ax[0].grid()
ax[1].grid()

for tick in ax[0].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[1].get_xticklabels():
    tick.set_fontsize(14)
for tick in ax[0].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[1].get_xticklabels():
    tick.set_fontsize(14)
ax[0].set_xlim([2019, 2045])
ax[1].set_xlim([2019, 2045])
ax[0].set_ylim([0,50])
ax[1].set_ylim([0,50])

handles = [
    plt.Line2D([], [], linestyle='solid', color='black', label = "BAU", linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[0], label='2025', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[1], label='2030', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[2], label='2035', linewidth = 2),
    plt.Line2D([], [], marker='', color=zy_cols[3], label='2040', linewidth = 2),    
]

handles2 = [
    plt.Line2D([], [], linestyle='solid', color='black', label = "BAU", linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[0], label='2025', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[1], label='2030', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[2], label='2035', linewidth = 2),
    plt.Line2D([], [], marker='', color=ry_cols[3], label='2040', linewidth = 2),    
]

handles3 = [
    plt.Line2D([], [], marker='', color="gray", linestyle = "solid",label='5 years', linewidth = 2),
    plt.Line2D([], [], marker='', color=r"gray", linestyle = "dashed", label='10 years', linewidth = 2),
    
]

l0 = ax[0].legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), title = "ZEV Mandate Year", fancybox = False, fontsize = 14)
l0.get_title().set_fontsize(14)
l1 = ax[1].legend(handles=handles2, loc='upper left', bbox_to_anchor=(1.02, 1), title = "Retirement Year", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)
l2 = ax[1].legend(handles=handles3, loc='upper left', bbox_to_anchor=(1.02, 0.5), title = "Retirement Age", fancybox = False, fontsize = 14)
l2.get_title().set_fontsize(14)
ax[1].add_artist(l1)

zev_mandate_emissions_e.plot(ax = ax[2], color = ["black","lightskyblue","paleturquoise","mediumturquoise","darkcyan"], linewidth = 2, legend = False)
retirements_emissions_e[2035][5].plot(ax = ax[3], color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2, legend = False)
retirements_emissions_e[2035][10].plot(ax = ax[3], linestyle = "dashed", color = ["black","gold","darkorange","orangered","firebrick"], linewidth = 2, legend = False)

ax[2].set_ylabel("Annual Electric CO2 Emissions \n(Million tonnes)", fontsize = 14)
ax[2].set_title("C) ZEV Sales Mandate Scenarios\n ", fontsize = 16)
ax[3].set_title("D) Accelerated Retirement Scenarios\nZEV Mandate Year = 2035", fontsize = 16)
ax[2].grid()
ax[3].grid()

for tick in ax[2].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[2].get_xticklabels():
    tick.set_fontsize(14)
for tick in ax[3].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[3].get_xticklabels():
    tick.set_fontsize(14)
ax[2].set_xlim([2019, 2045])
ax[3].set_xlim([2019, 2045])
ax[2].set_ylim([0,50])
ax[3].set_ylim([0,50])

plt.tight_layout()
f.savefig("annual_emissions_split.png")

In [ ]:
races = ["Latino","Black","Asian","White","Native","Pac Islander","Two or More"]
    
#     'White',
#  'Black',
#  'Native',
#  'Asian',
#  'Pac Islander',
#  'Two or More',
#  'Latino']

deaths_pc_race = pd.DataFrame(index = ["BAU",2040,2035,2030,2025,
        "2040-10-2040","2040-5-2040",
        "2040-10-2035","2040-5-2035",
        "2040-10-2030","2040-5-2030",
        "2040-10-2025","2040-5-2025",
        "2035-10-2040","2035-5-2040",
        "2035-10-2035","2035-5-2035",
        "2035-10-2030","2035-5-2030",
        "2035-10-2025","2035-5-2025"], 
                       columns = races)

In [ ]:
for r in races:
    d = 0
    for v in veh_types:
        d += results["BAU"][v]["Fossil"][f"{r}_deathsK_pc"].sum()
    deaths_pc_race.loc["BAU",r] = d
    for y in zev_years:
        d = 0
        for v in veh_types:
            d += results["ZEV_mandate"][y][v]["Fossil"][f"{r}_deathsK_pc"].sum()
        deaths_pc_race.loc[y,r] = d
    for s in results["retirements"].keys():
        d = 0
        for v in veh_types:
            d += results["retirements"][s][v]["Fossil"][f"{r}_deathsK_pc"].sum()
        deaths_pc_race.loc[s,r] = d
        


In [ ]:
f, ax = plt.subplots(1,1, figsize = (10,5))

deaths_pc_race.T[["BAU",2035,"2035-10-2040","2035-10-2035"]].plot.bar(ax = ax)

In [ ]:
incomes = ['<10,000', '10,000-14,999', '15,000 - 24,999', '25,000 - 34,999',
       '35,000 - 49,999', '50,000 - 74,999', '75,000 - 99,999',
       '100,000 - 149,999', '150,000 - 199,999', '>200,000']

In [ ]:
races = incomes
    
#     'White',
#  'Black',
#  'Native',
#  'Asian',
#  'Pac Islander',
#  'Two or More',
#  'Latino']

deaths_pc_income = pd.DataFrame(index = ["BAU",2040,2035,2030,2025,
        "2040-10-2040","2040-5-2040",
        "2040-10-2035","2040-5-2035",
        "2040-10-2030","2040-5-2030",
        "2040-10-2025","2040-5-2025",
        "2035-10-2040","2035-5-2040",
        "2035-10-2035","2035-5-2035",
        "2035-10-2030","2035-5-2030",
        "2035-10-2025","2035-5-2025"], 
                       columns = races)

In [ ]:
for r in incomes:
    d = 0
    for v in veh_types:
        d += results["BAU"][v]["Fossil"][f"{r}_deathsK_pc"].sum()
    deaths_pc_income.loc["BAU",r] = d
    for y in zev_years:
        d = 0
        for v in veh_types:
            d += results["ZEV_mandate"][y][v]["Fossil"][f"{r}_deathsK_pc"].sum()
        deaths_pc_income.loc[y,r] = d
    for s in results["retirements"].keys():
        d = 0
        for v in veh_types:
            d += results["retirements"][s][v]["Fossil"][f"{r}_deathsK_pc"].sum()
        deaths_pc_income.loc[s,r] = d

In [ ]:
races = ["Latino","Black","Asian","White","Native","Pac Islander","Two or More"]
deaths_pc_race_v = pd.DataFrame(index = races, columns = veh_types)
for r in races:
    for v in veh_types:
        deaths_pc_race_v.loc[r,v] = results["BAU"][v]["Fossil"][f"{r}_deathsK_pc"].sum()+results["BAU"][v]["Elec"][f"{r}_deathsK_pc"].sum()

In [ ]:
#races = ["Latino","Black","Asian","White","Native","Pac Islander","Two or More"]
deaths_pc_income_v = pd.DataFrame(index = incomes, columns = veh_types)
for r in incomes:
    for v in veh_types:
        deaths_pc_income_v.loc[r,v] = results["BAU"][v]["Fossil"][f"{r}_deathsK_pc"].sum()+results["BAU"][v]["Elec"][f"{r}_deathsK_pc"].sum()

In [ ]:
f, ax = plt.subplots(1,2, figsize = (15,10))
ax = ax.flatten()
deaths_pc_income_v.plot(ax = ax[0], marker = 'o', linestyle = "dashed", legend = False)
deaths_pc_race_v.plot(ax = ax[1], marker = 'o', linestyle = "dashed")
ax[0].set_ylabel("Cumulative deaths per capita\n(deaths per 100,000 ppl)", fontsize = 14)
l1 = ax[1].legend(loc='upper left', bbox_to_anchor=(1.02, 1), title = "Vehicle Type", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)
for tick in ax[0].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[0].get_xticklabels():
    tick.set_fontsize(14)
    tick.set_rotation(45)
for tick in ax[1].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[1].get_xticklabels():
    tick.set_fontsize(14)
    tick.set_rotation(45)
ax[0].grid()
ax[1].grid()
ax[0].set_title("A) Income", fontsize = 16)
ax[1].set_title("B) Race", fontsize = 16)
ax[0].set_ylim([0,5.5])
ax[1].set_ylim([0,5.5])
#f.savefig("race_income_deaths_pc_ca.png", bbox_inches = "tight")

In [ ]:
### Cost effectiveness relative to ZEV scenarios
cost_effectiveness = pd.DataFrame(columns = ["veh_type","ret_cost","co2_saved","deaths_saved","cost_per_tonne","cost_per_death","zev_year","ret_year","ret_age"])

for s in results_all_rets.keys():
    zev_year = int(s.split("-")[0])
    for v in veh_types:
        ret_cost = results_all_rets[s].loc[results_all_rets[s]["veh_type"]==v,"used_value"].sum()
        ret_cost_n = results_all_rets[s].loc[results_all_rets[s]["veh_type"]==v,"new_value"].sum()
        co2_saved = results_all[zev_year].loc[results_all[zev_year]["veh_type"]==v,"co2"].sum()-results_all_rets[s].loc[results_all_rets[s]["veh_type"]==v,"co2"].sum()
        deaths_saved = (results["ZEV_mandate"][zev_year][v]["Fossil"]["deathsK_CA"].sum()+ results["ZEV_mandate"][zev_year][v]["Fossil"]["deathsK_OCA"].sum()- 
                        results["retirements"][s][v]["Fossil"]["deathsK_CA"].sum() - results["retirements"][s][v]["Fossil"]["deathsK_OCA"].sum()+
                       results["ZEV_mandate"][zev_year][v]["Elec"]["deathsK_CA"].sum()+ results["ZEV_mandate"][zev_year][v]["Elec"]["deathsK_OCA"].sum()- 
                        results["retirements"][s][v]["Elec"]["deathsK_CA"].sum() - results["retirements"][s][v]["Elec"]["deathsK_OCA"].sum())
        cost_per_tonne = ret_cost/co2_saved
        cost_per_death = ret_cost/deaths_saved
        cost_per_tonne_n = ret_cost_n/co2_saved
        cost_per_death_n = ret_cost_n/deaths_saved
        ce = pd.DataFrame({"veh_type":v,
                                               "ret_cost":ret_cost/1e9,
                                               "ret_cost_n":ret_cost_n/1e9,
                                               "co2_saved":co2_saved/1e6,
                                               "deaths_saved":deaths_saved,
                                               "cost_per_tonne":cost_per_tonne,
                                               "cost_per_death":cost_per_death/1e6,
                                               "zev_year":s.split("-")[0],
                                               "ret_year":s.split("-")[2],
                                               "ret_age":s.split("-")[1]}, index = [0])
        cost_effectiveness = cost_effectiveness.append(ce, ignore_index = True)
cost_effectiveness["zev_year"] = cost_effectiveness["zev_year"].astype(int)
cost_effectiveness["ret_year"] = cost_effectiveness["ret_year"].astype(int)
cost_effectiveness["ret_age"] = cost_effectiveness["ret_age"].astype(int)
 

In [ ]:
veh_colors = {
    "T6": "royalblue",
    "T6_OOS": "lightcoral",
    "T7": "darkred",
    "T7_OOS": "darkorange",
    "T7_Port": "lightseagreen",
    "LHD1": "darkslategrey",
    "LHD2": "plum",
    "MC": "lightpink",
    "MH": "darkgoldenrod",
    "Buses": "skyblue"
    
}

In [ ]:
ra_shapes = {5:"o",10:"^"}

In [ ]:
for v in veh_types:
    cost_effectiveness.loc[cost_effectiveness.veh_type==v,"color"] = veh_colors[v]
for ra in [5,10]:
    cost_effectiveness.loc[cost_effectiveness.ret_age==ra,"marker"] = ra_shapes[ra]
   

In [ ]:
for i,ry in enumerate([2025,2030,2035,2040]):
    for j,zy in enumerate([2035,2040]):
        cost_effectiveness.loc[(cost_effectiveness.ret_year==ry) & (cost_effectiveness.zev_year == zy),"x_pos"] = i +j*4

In [ ]:
new_price = pd.read_csv("hdv_bev_prices.csv")

In [ ]:
cost_effectiveness["co2_value"] = cost_effectiveness["co2_saved"]*51/1e3
cost_effectiveness["deaths_value"] = cost_effectiveness["deaths_saved"]*9.63e6/1e9
cost_effectiveness["total_value"] = cost_effectiveness["co2_value"]+cost_effectiveness["deaths_value"] #in billions of dollars

In [ ]:
f, ax = plt.subplots(1,2, figsize = (15,8))


cost_effectiveness["co2_value"] = cost_effectiveness["co2_saved"]*51/1e3
cost_effectiveness["deaths_value"] = cost_effectiveness["deaths_saved"]*9.63e6/1e9
cost_effectiveness["total_value"] = cost_effectiveness["co2_value"]+cost_effectiveness["deaths_value"] #in billions of dollars
cost_effectiveness["co2_value_low"] = cost_effectiveness["co2_saved"]*51/1e3
cost_effectiveness["total_value_low"] = cost_effectiveness["co2_value_low"]+cost_effectiveness["deaths_value"] #in billions of dollars


for i,z in enumerate(ret_years):
    ax[0].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==5) & (cost_effectiveness.ret_year==z)].ret_cost, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].color,
                 marker = "^", s = 50, alpha = 1-i*0.25)
    ax[0].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].ret_cost, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].color,
                 marker = "o", s = 50,alpha = 1-i*0.25)
x = np.linspace(0,27,5)
y = x
ax[0].plot(x,y, linestyle = "dashed",color = "black")
ax[0].set_xlim(0,27)
ax[0].set_ylim(0,27)

cost_effectiveness["co2_value"] = cost_effectiveness["co2_saved"]*190/1e3
cost_effectiveness["deaths_value"] = cost_effectiveness["deaths_saved"]*9.63e6/1e9
cost_effectiveness["total_value"] = cost_effectiveness["co2_value"]+cost_effectiveness["deaths_value"] #in billions of dollars
for i,z in enumerate(ret_years):
    ax[0].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==5) & (cost_effectiveness.ret_year==z)].ret_cost, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].color,
                 marker = "^", s = 50, alpha = 1-i*0.25)
    ax[0].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].ret_cost, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].color,
                 marker = "o", s = 50,alpha = 1-i*0.25)
    ax[0].vlines(cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_year==z)].ret_cost,
                cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_year==z)].total_value_low,
                cost_effectiveness[(cost_effectiveness.zev_year==2035)&  (cost_effectiveness.ret_year==z)].total_value,
                color = cost_effectiveness[(cost_effectiveness.zev_year==2035)&  (cost_effectiveness.ret_year==z)].color,
                 alpha = (1-i*0.25)*0.25, linestyle = "solid", linewidth = 5)
x = np.linspace(0,27,5)
y = x
ax[0].plot(x,y, linestyle = "dashed",color = "black")
ax[0].set_xlim(0,27)
ax[0].set_ylim(0,27)


cost_effectiveness["co2_value"] = cost_effectiveness["co2_saved"]*51/1e3
cost_effectiveness["deaths_value"] = cost_effectiveness["deaths_saved"]*9.63e6/1e9
cost_effectiveness["total_value"] = cost_effectiveness["co2_value"]+cost_effectiveness["deaths_value"] #in billions of dollars


for i,z in enumerate(ret_years):
    ax[1].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==5) & (cost_effectiveness.ret_year==z)].ret_cost_n, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].color,
                 marker = "^", s = 50, alpha = 1-i*0.25)
    ax[1].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].ret_cost_n, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].color,
                 marker = "o", s = 50,alpha = 1-i*0.25)
x = np.linspace(0,85,5)
y = x
ax[1].plot(x,y, linestyle = "dashed",color = "black")
ax[1].set_xlim([0,85])
ax[1].set_ylim([0,85])

cost_effectiveness["co2_value"] = cost_effectiveness["co2_saved"]*190/1e3
cost_effectiveness["deaths_value"] = cost_effectiveness["deaths_saved"]*9.63e6/1e9
cost_effectiveness["total_value"] = cost_effectiveness["co2_value"]+cost_effectiveness["deaths_value"] #in billions of dollars


for i,z in enumerate(ret_years):
    ax[1].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==5) & (cost_effectiveness.ret_year==z)].ret_cost_n, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==5)& (cost_effectiveness.ret_year==z)].color,
                 marker = "^", s = 50, alpha = 1-i*0.25)
    ax[1].scatter(x = cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].ret_cost_n, 
               y = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].total_value,
                 color = cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_age==10)& (cost_effectiveness.ret_year==z)].color,
                 marker = "o", s = 50,alpha = 1-i*0.25)
    ax[1].vlines(cost_effectiveness[(cost_effectiveness.zev_year==2035) & (cost_effectiveness.ret_year==z)].ret_cost_n,
                cost_effectiveness[(cost_effectiveness.zev_year==2035)& (cost_effectiveness.ret_year==z)].total_value_low,
                cost_effectiveness[(cost_effectiveness.zev_year==2035)&  (cost_effectiveness.ret_year==z)].total_value,
                color = cost_effectiveness[(cost_effectiveness.zev_year==2035)&  (cost_effectiveness.ret_year==z)].color,
                 alpha = (1-i*0.25)*0.25, linestyle = "solid", linewidth = 5)
    
ax[0].set_ylabel("Cumulative Avoided Health and Climate Damages\n($Billion)", fontsize = 14)
ax[0].set_xlabel("Cumulative Cost of Early Retirements\nUsing Second-hand Market Price\n($Billion)", fontsize = 14)
ax[1].set_xlabel("Cumulative Cost of Early Retirements\nUsing New Electric Vehicle Price\n($Billion)", fontsize = 14)
ax[0].text(15,22,"Net Benefit", fontsize = 14)
ax[0].text(22,20,"Net Cost", fontsize = 14)

ax[1].text(47,69,"Net Benefit", fontsize = 14)
ax[1].text(69,63,"Net Cost", fontsize = 14)


for tick in ax[0].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[0].get_xticklabels():
    tick.set_fontsize(14)
    
for tick in ax[1].get_yticklabels():
    tick.set_fontsize(14)
for tick in ax[1].get_xticklabels():
    tick.set_fontsize(14)

handles = [mpatches.Patch(facecolor=veh_colors[v], hatch='', edgecolor=veh_colors[v], label = veh_types[i]) for i,v in enumerate(veh_types)]
l0 = ax[1].legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), title = "Vehicle Type", fancybox = False, fontsize = 14)
l0.get_title().set_fontsize(14)

handles2 = [
    plt.Line2D([], [], linestyle='none', color='black', label = "5", marker = "^", markersize = 7),
    plt.Line2D([], [], linestyle='none', color='black', label = "10", marker = "o", markersize = 7)
]

handles3 = [plt.Line2D([], [], linestyle='none', color='gray', label = ret_years[0], marker = "s", markersize = 7, alpha = 1),
           plt.Line2D([], [], linestyle='none', color='gray', label = ret_years[1], marker = "s", markersize = 7, alpha = 1-1*0.25),
           plt.Line2D([], [], linestyle='none', color='gray', label = ret_years[2], marker = "s", markersize = 7, alpha = 1-2*0.25),
           plt.Line2D([], [], linestyle='none', color='gray', label = ret_years[3], marker = "s", markersize = 7, alpha = 1-3*0.25)]

l1 = ax[1].legend(handles=handles2, loc='upper left', bbox_to_anchor=(1.02, 0.23), title = "Retirement Age", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)

l2 = ax[1].legend(handles=handles3, loc='upper left', bbox_to_anchor=(1.02, 0.48), title = "Retirement Year", fancybox = False, fontsize = 14)
l2.get_title().set_fontsize(14)
ax[1].add_artist(l1)
ax[1].add_artist(l0)
#ax.set_title("SCC = $190/tonne\nNew BEV Price", fontsize = 14)

ax[0].text(1,27.5,"(A)", fontsize = 14)
ax[1].text(3.2,86.5,"(B)", fontsize = 14)
plt.tight_layout()

f.savefig("cost_plot_all.png", bbox_inches = "tight")


In [ ]:
cum_deaths = pd.DataFrame(index = ["BAU",2040,2035,2030,2025,
        "2040-10-2040","2040-5-2040",
        "2040-10-2035","2040-5-2035",
        "2040-10-2030","2040-5-2030",
        "2040-10-2025","2040-5-2025",
        "2035-10-2040","2035-5-2040",
        "2035-10-2035","2035-5-2035",
        "2035-10-2030","2035-5-2030",
        "2035-10-2025","2035-5-2025"], 
                       columns = ["cum_deaths","zev_year","ret_year","ret_age","color","outline_color","hatch"])

In [ ]:
cum_deaths.loc["BAU","cum_deaths"] = 0
for v in veh_types:
    cum_deaths.loc["BAU","cum_deaths"]+=(results["BAU"][v]["Fossil"]["deathsK_CA"].sum()+
                                         results["BAU"][v]["Fossil"]["deathsK_OCA"].sum()+
                                        results["BAU"][v]["Elec"]["deathsK_CA"].sum()+
                                         results["BAU"][v]["Elec"]["deathsK_OCA"].sum())
cum_deaths.loc["BAU","zev_year"] = 100
cum_deaths.loc["BAU","ret_year"] = 100
cum_deaths.loc["BAU","ret_age"] = 100

for y in [2025, 2030, 2035, 2040]:
    cum_deaths.loc[y,"cum_deaths"] = 0
    for v in veh_types:
        cum_deaths.loc[y,"cum_deaths"] += (results["ZEV_mandate"][y][v]["Fossil"]["deathsK_CA"].sum()+
                                           results["ZEV_mandate"][y][v]["Fossil"]["deathsK_OCA"].sum()+
                                          results["ZEV_mandate"][y][v]["Elec"]["deathsK_CA"].sum()+
                                           results["ZEV_mandate"][y][v]["Elec"]["deathsK_OCA"].sum())
    cum_deaths.loc[y,"zev_year"] = y
    cum_deaths.loc[y,"ret_year"] = 100
    cum_deaths.loc[y,"ret_age"] = 100
for s in results_all_rets.keys():
    cum_deaths.loc[s,"cum_deaths"] = 0
    for v in veh_types:
        cum_deaths.loc[s,"cum_deaths"]+=(results["retirements"][s][v]["Fossil"]["deathsK_CA"].sum()+
                                         results["retirements"][s][v]["Fossil"]["deathsK_OCA"].sum()+
                                        results["retirements"][s][v]["Elec"]["deathsK_CA"].sum()+
                                         results["retirements"][s][v]["Elec"]["deathsK_OCA"].sum())
    cum_deaths.loc[s,"zev_year"] = int(s.split("-")[0])
    cum_deaths.loc[s,"ret_year"] = int(s.split("-")[2])
    cum_deaths.loc[s,"ret_age"] = int(s.split("-")[1])

cum_deaths["ind_str"] = cum_deaths.index
cum_deaths["ind_str"] = cum_deaths["ind_str"].astype("str")

zy_cols = ["lightskyblue","paleturquoise","mediumturquoise","darkcyan"]#["gold","thistle","mediumturquoise","lightcoral"]
ry_cols = ["gold","darkorange","orangered","firebrick"]#["midnightblue","orange","brown","pink"]
ra_hatch = ["//","o"]

cum_deaths.loc["BAU","color"] = "gray"
cum_deaths.loc["BAU","hatch"] = ""
cum_deaths.loc["BAU","outline_color"] = "gray"
for i,zy in enumerate([2025, 2030,2035,2040]):
    cum_deaths.loc[cum_deaths.zev_year==zy,"color"] = zy_cols[i]
    cum_deaths.loc[cum_deaths.zev_year==zy,"hatch"] = ""
    cum_deaths.loc[cum_deaths.zev_year==zy,"outline_color"] = zy_cols[i] 
for i,ry in enumerate([2025,2030,2035,2040]):
    print(i,ry)
    cum_deaths.loc[cum_deaths.ret_year==ry,"outline_color"] = ry_cols[i]
for i, ra in enumerate([5,10]):
    cum_deaths.loc[cum_deaths.ret_age==ra,"hatch"] = ra_hatch[i]


In [ ]:
f, ax = plt.subplots(1,1, figsize = (20,8))


ax.bar(x= cum_deaths.ind_str, height = cum_deaths.cum_deaths,color = cum_deaths.color, 
       hatch = cum_deaths.hatch, edgecolor = cum_deaths.outline_color, linewidth = 3)
ax.vlines(0.5,0,6000, linestyles = "dashed", color = "black")
ax.vlines(4.5,0,6000, linestyles = "dashed", color = "black")
ax.set_ylim([0,6000])
plt.xticks(rotation = 45)
ax.text(-1,5700,"BAU", fontsize = 14)
ax.text(0.7,5700,"ZEV Mandate Scenarios", fontsize = 14)
ax.text(10,5700,"Accelerated Retirement Scenarios", fontsize = 14)
ax.set_ylabel("Cumulative Deaths", fontsize = 16)
ax.text(7,4300,"ZEV Mandate in 2040", fontsize = 14)
ax.text(16,4300,"ZEV Mandate in 2035", fontsize = 14)


handles = [
    mpatches.Patch(facecolor=zy_cols[0], hatch='', edgecolor=zy_cols[0], label = "2025"),
    mpatches.Patch(facecolor=zy_cols[1], hatch='', edgecolor=zy_cols[1], label = "2030"),
    mpatches.Patch(facecolor=zy_cols[2], hatch='', edgecolor=zy_cols[2], label = "2035"),
    mpatches.Patch(facecolor=zy_cols[3], hatch='', edgecolor=zy_cols[3], label = "2040")
]

handles2 = [
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[0], label = "2025"),
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[1], label = "2030"),
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[2], label = "2035"),
    mpatches.Patch(facecolor="white", hatch='', edgecolor=ry_cols[3], label = "2040")
]

handles3 = [
    mpatches.Patch(facecolor="white", hatch=ra_hatch[0], edgecolor="black", label = "5 years"),
    mpatches.Patch(facecolor="white", hatch=ra_hatch[1], edgecolor = "black", label = "10 years")
]

# handles4 = [
#     mpatches.Patch(facecolor="gray", hatch='', edgecolor="gray", label = "ICE Tailpipe Emissions"),
#     mpatches.Patch(facecolor="gray", hatch='', edgecolor="gray", alpha = 0.5, label = "Electricity Generation Emissions")
# ]


zy_cols = ["lightskyblue","paleturquoise","mediumturquoise","darkcyan"]#["gold","thistle","mediumturquoise","lightcoral"]
ry_cols = ["gold","darkorange","orangered","firebrick"]#["midnightblue","orange","brown","pink"]
ra_hatch = ["//","o"]




# handles = [
#     mpatches.Patch(facecolor='red', hatch='/', edgecolor='black', label = "2025"),
#     mpatches.Patch(facecolor='green', hatch='\\', edgecolor='black'),
#     mpatches.Patch(facecolor='blue', hatch='|', edgecolor='black'),
#     mpatches.Patch(facecolor='yellow', hatch='-', edgecolor='black'),
#     plt.Line2D([], [], linestyle='dashed', color='black'),
#     plt.Line2D([], [], marker='', color='white', label='BAU'),
#     plt.Line2D([], [], marker='', color='white', label='ZEV Mandate Scenarios'),
#     plt.Line2D([], [], marker='', color='white', label='Accelerated Retirement Scenarios')
# ]

# Create the legend and add it to the plot
# l1.get_title().set_ha('left')
l2 = ax.legend(handles=handles2, loc='upper left', bbox_to_anchor=(1.02, 0.75), title = "Retirement Year", fancybox = False, fontsize = 14)
l2.get_title().set_fontsize(14)
# l2.get_title().set_ha('left')
l3 = ax.legend(handles=handles3, loc='upper left', bbox_to_anchor=(1.02, 0.5), title = "Retirement Age", fancybox = False, fontsize = 14)
l3.get_title().set_fontsize(14)
# l3.get_title().set_ha('left')
# l4 = ax.legend(handles=handles4, loc='upper left', bbox_to_anchor=(1.02, 1), title = "Emissions Type", fancybox = False, fontsize = 14)
# l4.get_title().set_fontsize(14)
# l4.get_title().set_ha('left')
l1 = ax.legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), title = "ZEV Sales Mandate Year", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)



ax.add_artist(l3)
ax.add_artist(l2)
# ax.add_artist(l3)


for tick in ax.get_yticklabels():
    tick.set_fontsize(14)
for tick in ax.get_xticklabels():
    tick.set_fontsize(14)
    
ax.text(1, -1500, "ZEV Mandate Year", fontsize = 14)
ax.text(8.5, -1500,"ZEV mandate year - retirement age - retirement year", fontsize = 14)

plt.tight_layout()
#f.savefig("cumulative_deaths_scen.png", bbox_inches = "tight")

# plt.gca().add_artist(l1)
# plt.gca().add_artist(l2)

In [ ]:
deaths_map = pm25_fossil["T6"].copy()
for v in ["T6_OOS","T7","T7_OOS","T7_Port","LHD1","LHD2","MC","MH","Buses"]:
    deaths_map["TotalPM25"]+=pm25_fossil[v]["TotalPM25"]
    deaths_map["deathsK"]+=pm25_fossil[v]["deathsK"]
deaths_map["deathsK_pc"] = deaths_map["deathsK"]/deaths_map["Population"]*100000

In [ ]:
deaths_map_d = attach_demographics(deaths_map, demographics)

In [ ]:
deaths_map_ca = deaths_map_d[deaths_map_d.STATEFP=="06"]

In [ ]:
counties = gpd.read_file("tl_2019_us_county.shp")

In [ ]:
f, ax = plt.subplots(2,5, figsize = (20,10))
ax = ax.flatten()
for i,v in enumerate(veh_types):    
    counties[counties.STATEFP=="06"].boundary.plot(color = "gray", ax = ax[i], linewidth = 0.5)
    df = attach_demographics(pm25_fossil[v], demographics)
    df = df[df.STATEFP=="06"]
    df["deathsK_pc"] = df["deathsK"]/df["Population"]*1000000
    vmax = 1#np.ceil(df["TotalPM25"].quantile([0.75])[0.75])
    df.plot(column = "TotalPM25", vmax = 1, cmap = "Reds", ax = ax[i], 
                   norm = colors.LogNorm(vmin = 1e-3, vmax = vmax), edgecolor="face", linewidth=0.4)
    ax[i].axis("off")
    ax[i].set_title(v, fontsize = 16)
    
sm = plt.cm.ScalarMappable(cmap = "Reds", norm = colors.LogNorm(vmin = 1e-3, vmax = vmax))
cax = f.add_axes([0.97, 0.15, 0.02, 0.75])
cbar = f.colorbar(sm, cax = cax, extend = "both")
cbar.ax.tick_params(labelsize = 15)
cbar.set_label("Increase in PM2.5 Concentration (ug/m3)", fontsize = 18)   
f.savefig("pm25_2019_all.png", bbox_inches = "tight")

In [ ]:
bau_stock = {}
for f in ["Diesel","Gasoline","Natural Gas","Electricity"]:
    bau_stock[f] = BAU_all[BAU_all.fuel_type==f].groupby(["year",
                                                          "veh_type"]).agg({"stock":"sum"}).reset_index().pivot(
        index = "year", columns = ["veh_type"], values = "stock")
    bau_stock[f] = bau_stock[f]/1000

In [ ]:
f, ax = plt.subplots(2,5, figsize = (15,6))
ax = ax.flatten()

letters = ["A","B","C","D","E","F","G","H","I","J"]
for i, v in enumerate(veh_types):
    ax[i].bar(bau_stock["Diesel"].index, bau_stock["Diesel"][v], label = "Diesel", width = 0.9)
    ax[i].bar(bau_stock["Gasoline"].index, bau_stock["Gasoline"][v], bottom = bau_stock["Diesel"][v], label = "Gasoline", width = 0.9)
    ax[i].bar(bau_stock["Natural Gas"].index, bau_stock["Natural Gas"][v], bottom = bau_stock["Diesel"][v]+bau_stock["Gasoline"][v],
      label = "Natural Gas", width = 0.9)
    ax[i].bar(bau_stock["Electricity"].index, bau_stock["Electricity"][v], bottom = bau_stock["Diesel"][v]+bau_stock["Gasoline"][v]+
      bau_stock["Natural Gas"][v], label = "Electricity", width = 0.9)
    #l1 = ax[i].legend(loc='upper left', bbox_to_anchor=(1.02, 1), title = "Fuel Type", fancybox = False, fontsize = 14)
    

    for tick in ax[i].get_yticklabels():
        tick.set_fontsize(14)
    for tick in ax[i].get_xticklabels():
        tick.set_fontsize(14)
    # ax[i].set_ylabel("Vehicle Stock", fontsize = 14)
    ax[i].set_title(f"{letters[i]}) {v}",fontsize = 16)


f.supylabel("Vehicle Stock\n(Thousands)", fontsize = 14, ha = "center")
l1 = ax[4].legend(loc='upper left', bbox_to_anchor=(1.02, 1), title = "Fuel Type", fancybox = False, fontsize = 14)
l1.get_title().set_fontsize(14)

plt.tight_layout()
f.savefig("veh_stock_bau.pdf", bbox_inches = "tight")

In [ ]:
p =BAU_all.loc[(BAU_all.year==2019) & (BAU_all.veh_type=="T6"),["age","stock","fuel_type"]].pivot(index = "age", columns = "fuel_type", values = "stock")

In [ ]:
for v in veh_types:
    f, ax = plt.subplots(1,1,figsize = (10,5))
    p =BAU_all.loc[(BAU_all.year==2019) & (BAU_all.veh_type==v),
                   ["age","stock","fuel_type"]].pivot(index = "age", 
                                                      columns = "fuel_type", 
                                                      values = "stock")[["Diesel","Gasoline","Natural Gas","Electricity"]]
    p = p/1000
    p.plot(
        kind = "bar", stacked = True, ax = ax)
    l = ax.legend(title = "Fuel Type", fontsize = 14)
    l.get_title().set_fontsize (14)
    ax.set_xlabel("Vehicle Age", fontsize = 14)
    ax.set_ylabel ("Vehicle Stock in 2019 \n(Thousands)", fontsize = 14)
    ax.set_xlim([-0.5,44.5])
    
    for tick in ax.get_xticklabels():
        tick.set_fontsize(14)
        tick.set_rotation(90)
    for tick in ax.get_yticklabels():
        tick.set_fontsize(14)
    ax.set_title(v,fontsize = 16)
    
    f.savefig(f"init_vintage_{v}.png")
